In [0]:
#### AR Model ####
## This script is used to calculate account receivable allocation for different levels per user selection. 
## Datasets are from Product ID level granular data and 7 bucket aging Invoice level data, which read from Blob storage and ADX table. Flat file includes carve out percentage for service PIDs. Tagging files are used to tag PID as in scope/out of scope/product/service and other categories for breakdown analysis. 
## To calculate allocation percentage, using Product ID data unit price divided by total unit price. Then multiply the percentage to outstanding amount from Invoice level data to get final AR allocation amount. If there is no match between two datasets at a certain level, will proceed to a higher level for allocation calculation. 
## All dollar amounts are in local currency.
## Parameters have been set up, so user can select allocation level, columns used to join for each dataset, and any specific condition may need to be applied from top drop down menu. 
## If not using Default cluster, need to install azure package first, in order to build ADX connection. 
## Data cleaning major part has been done in Alteryx in advance.

In [0]:
## Import libraries 
from pyspark.sql.functions import col, lit, trim, substring, concat, when
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window
from pyspark.sql import functions as F
import pandas as pd 
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType, DecimalType
from pyspark.sql.functions import collect_set,sum,avg,max,countDistinct,count
import threading
from threading import Thread

# show pyspark dataframe in pandas format
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [0]:
# Check files in Capital Edge shared files folder
dir_path = '/mnt/sources/'
src_files = []
def walk_dir(dir_path):
  dir_files = dbutils.fs.ls(dir_path)
  for file in dir_files:
    if file.isDir():
      walk_dir(file.path)
    else: src_files.append(file.path)

In [0]:
walk_dir('/mnt/sources/')
src_files

Out[3]: ['dbfs:/mnt/sources/6051a7a4b3c52f3187ce6a54/6051a8deb3c52f55a9ce6a55/AS and TS Data.csv',
 'dbfs:/mnt/sources/6051ae51b3c52fa1c8ce6a56/6051ae5cb3c52f250ece6a57/Products Bookings.csv',
 'dbfs:/mnt/sources/6051b3beb3c52f419ace6a58/6051b3c8b3c52f10dace6a59/Products Revenue.csv',
 'dbfs:/mnt/sources/6051b3beb3c52f419ace6a58/6051be86b3c52f6c4ece6a5c/Products Revenue.csv',
 'dbfs:/mnt/sources/6051bd3fb3c52f89e8ce6a5a/6051bd44b3c52fda01ce6a5b/Lookup PIDs.csv',
 'dbfs:/mnt/sources/6051d46fb3c52f32e0ce6a5d/6051d479b3c52f2400ce6a5e/Consolidated Rev_COGS_Part1.csv',
 'dbfs:/mnt/sources/6051d46fb3c52f32e0ce6a5d/6051dc22b3c52f0734ce6a5f/Consolidated Rev_COGSPart2.csv',
 'dbfs:/mnt/sources/6051e281b3c52fd7eece6a60/6051e286b3c52f68e4ce6a61/Out Of Scope PIDs.csv',
 'dbfs:/mnt/sources/6051ec44b3c52f74aace6a62/6051ec48b3c52fde23ce6a63/Lookup PIDs.csv',
 'dbfs:/mnt/sources/60bf231db3c52faa42ce6a64/60bf2329b3c52fb264ce6a65/Consolidate AS_0706.csv',
 'dbfs:/mnt/sources/60bf23d7b3c52f2c45ce6a66/60bf23deb3c52ff15ace6a67/Consolidate TS_0706.csv',
 'dbfs:/mnt/sources/60bf5944b3c52f65edce6a68/60bf5949b3c52f8eedce6a69/Consolidated Computing Business Booking_0706.csv',
 'dbfs:/mnt/sources/60bf5a78b3c52f75f7ce6a6a/60bf5a7db3c52f3005ce6a6b/Consolidated Rev_COGS_0706.csv',
 'dbfs:/mnt/sources/60bf5a78b3c52f75f7ce6a6a/60bf6dbcb3c52f8addce6a6c/Part1.csv',
 'dbfs:/mnt/sources/60bf5a78b3c52f75f7ce6a6a/60bf77f1b3c52f4e89ce6a6d/Part2.csv',
 'dbfs:/mnt/sources/60cc4599708255ee347f57ff/60cc459b708255e44b7f5800/DealCustomerMapping.csv',
 'dbfs:/mnt/sources/60cc8126708255bc5b7f5802/60cc812a70825521007f5803/CiscoTBData.csv',
 'dbfs:/mnt/sources/60cc8126708255bc5b7f5802/60d2d4227082552ba17f5807/CiscoTBData_Qtr.csv',
 'dbfs:/mnt/sources/60cc8126708255bc5b7f5802/60d2e7cd70825515287f5808/CiscoTBData_Qtr.csv',
 'dbfs:/mnt/sources/60cc8126708255bc5b7f5802/60d2fb1970825525447f5809/CiscoTBData_Qtr.csv',
 'dbfs:/mnt/sources/60cc8126708255bc5b7f5802/60d5664b708255ba2d7f580e/CiscoTBData_Qtr.csv',
 'dbfs:/mnt/sources/60cc8126708255bc5b7f5802/6131d60bedfaf81337264027/CiscoTBData_Qtr_JUL21.csv',
 'dbfs:/mnt/sources/60cc8126708255bc5b7f5802/6131ffbdedfaf8697826402a/CiscoTBData_Qtr_JUL21.csv',
 'dbfs:/mnt/sources/60cc8126708255bc5b7f5802/61963293d08c848253444131/CiscoTBData_Qtr_OCT22.csv',
 'dbfs:/mnt/sources/60d9a0247082550eff7f5814/60d9a04870825500047f5815/Sheet2.csv',
 'dbfs:/mnt/sources/60d9a0247082550eff7f5814/60d9a04870825500047f5815/TMS Split.xlsx',
 'dbfs:/mnt/sources/60d9a3b970825508a07f5816/60d9a3c170825520f57f5817/PF and PID mapping.xlsx',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/60def326708255193d7f5823/Test JE 1.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/60def354708255576e7f5824/Test JE 2.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/60df41ae7082555abf7f5825/Test1.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/61018ead7082555bb37f5855/Test Entry.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/610294017082558a867f5869/Test JE 3.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/610299237082551c527f586a/Test JE 4.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/6102ad8d708255a8c57f586b/Test JE 5.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/6102bc6f70825514977f586c/Test JE 6.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/6102e21870825541e07f5876/Test Entry.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/615d53b3edfaf8154c264133/Test New 1.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/615d5439edfaf8606d264134/Test New 2.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/615d54c4edfaf80d7d264135/Test New 3.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/615d5b46edfaf83425264136/Test 5.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/615d5c6cedfaf8c621264137/Test New 6.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/616011dfedfaf885e926414f/Test New 7.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/616425caedfaf81079264150/Test 10112021.csv',
 'dbfs:/mnt/sources/60def0c4708255422f7f5822/61642e4fedfaf831fc264152/Te

In [0]:
# Create Allocation level input parameter
Level1 = "L1: Invoice & Type"
Level2 = "L2: Customer"
Level3 = "L3: Entity"

dbutils.widgets.dropdown("Allocation Level", Level1, [Level1, Level2, Level3])

dbutils.widgets.get("Allocation Level")

Out[4]: 'L3: Entity'

In [0]:
# pip install azure-functions

In [0]:
# pip install azure-functions

In [0]:
# Get credentials from adx
adxTenantId = dbutils.secrets.get(scope = "ce5", key = "adxTenantId")
adxClientId = dbutils.secrets.get(scope = "ce5", key = "adxClientId")
adxClientSecret = dbutils.secrets.get(scope = "ce5", key = "adxClientSecret")
adxClusterName = dbutils.secrets.get(scope = "ce5", key = "adxClusterName")
adxDatabaseName = dbutils.secrets.get(scope = "ce5", key = "adxDatabaseName")

In [0]:
# Establish connection
from pyspark.sql import SparkSession

pyKusto = SparkSession.builder.appName("kustoPySpark").getOrCreate()

kustoOptions = {\
  "kustoCluster": adxClusterName,\
  "kustoDatabase": adxDatabaseName,\
  "kustoAadAppId": adxClientId,\
  "kustoAadAppSecret": adxClientSecret,\
  "kustoAadAuthorityID": adxTenantId\
}

In [0]:
# Read data from adx table
query = "ARPIDFY21Clean2"  # change adx table name here

PID_raw  = spark.read. \
  format("com.microsoft.kusto.spark.datasource"). \
  option("kustoCluster", kustoOptions["kustoCluster"]). \
  option("kustoDatabase", kustoOptions["kustoDatabase"]). \
  option("kustoQuery", query). \
  option("kustoAadAppId", kustoOptions["kustoAadAppId"]). \
  option("kustoAadAppSecret", kustoOptions["kustoAadAppSecret"]). \
  option("kustoAadAuthorityID", kustoOptions["kustoAadAuthorityID"]). \
  load()

display(PID_raw)

ENTTRXCUSTTYPE,EntityNumberSource,Customer_number,TRX_NUMBER,Sales_Order,Transaction_Type,UnitPriceforPID,PID,PIDDescription,EntityNumberAccount,iris_id,iris_metadata
0200209597154436823INV,020,36823,95971544,112697039,INV,1354.74,PWR-C1-715WAC-P/2,715W AC 80+ platinum Config 1 SecondaryPower Supply,020,46f03d75-492c-4574-bc38-dd57d39633c1,"{ ""uploadedBy"": { ""name"": ""Fortuna Zhang"", ""id"": ""c8345eb7-e19e-4d32-ac91-52f61fcf1f8d"" }, ""uploadedOn"": ""2021-11-05T23:24:55.0820000Z"", ""uploadId"": ""6185bd47d08c84086a4440b3"", ""filename"": ""ARPIDFY21Clean2.csv"" }"
0200209597154436823INV,020,36823,95971544,112697039,INV,108.36,STACK-T1-50CM,50CM Type 1 Stacking Cable,020,c354a3c2-7fc5-41fe-92fb-2a275f25b0c4,"{ ""uploadedBy"": { ""name"": ""Fortuna Zhang"", ""id"": ""c8345eb7-e19e-4d32-ac91-52f61fcf1f8d"" }, ""uploadedOn"": ""2021-11-05T23:24:55.0820000Z"", ""uploadId"": ""6185bd47d08c84086a4440b3"", ""filename"": ""ARPIDFY21Clean2.csv"" }"
0200209597154436823INV,020,36823,95971544,112697039,INV,2763.66,C9300-NM-8X,Catalyst 9300 8 x 10GE Network Module,020,55211e52-1103-4ff9-b858-089b8ac73951,"{ ""uploadedBy"": { ""name"": ""Fortuna Zhang"", ""id"": ""c8345eb7-e19e-4d32-ac91-52f61fcf1f8d"" }, ""uploadedOn"": ""2021-11-05T23:24:55.0820000Z"", ""uploadId"": ""6185bd47d08c84086a4440b3"", ""filename"": ""ARPIDFY21Clean2.csv"" }"
0200209597154436823INV,020,36823,95971544,112697039,INV,10841.28,C9300-48P-E,"Catalyst 9300 48-port PoE+, Network Essentials",020,d2099d86-7b78-4e59-8aae-74b3a1bb137f,"{ ""uploadedBy"": { ""name"": ""Fortuna Zhang"", ""id"": ""c8345eb7-e19e-4d32-ac91-52f61fcf1f8d"" }, ""uploadedOn"": ""2021-11-05T23:24:55.0820000Z"", ""uploadId"": ""6185bd47d08c84086a4440b3"", ""filename"": ""ARPIDFY21Clean2.csv"" }"
0200209597154536823INV,020,36823,95971545,112802917,INV,719.3,C9300-DNA-E-48-3Y,"C9300 DNA Essentials, 48-port - 3 Year Term License",020,c1d92e2b-e602-4798-b02a-e62d3bac86aa,"{ ""uploadedBy"": { ""name"": ""Fortuna Zhang"", ""id"": ""c8345eb7-e19e-4d32-ac91-52f61fcf1f8d"" }, ""uploadedOn"": ""2021-11-05T23:24:55.0820000Z"", ""uploadId"": ""6185bd47d08c84086a4440b3"", ""filename"": ""ARPIDFY21Clean2.csv"" }"
0200209597154536823INV,020,36823,95971545,112802917,INV,719.3,C9300-DNA-E-48-3Y,"C9300 DNA Essentials, 48-port - 3 Year Term License",020,b151bfc4-98c5-441a-a4b3-4b7bc74ae94a,"{ ""uploadedBy"": { ""name"": ""Fortuna Zhang"", ""id"": ""c8345eb7-e19e-4d32-ac91-52f61fcf1f8d"" }, ""uploadedOn"": ""2021-11-05T23:24:55.0820000Z"", ""uploadId"": ""6185bd47d08c84086a4440b3"", ""filename"": ""ARPIDFY21Clean2.csv"" }"
0200209597154636823INV,020,36823,95971546,112752910,INV,2595.4,UCSC-PCIE-QD16GF=,Qlogic QLE2692 dual-port 16G FC HBA,020,e6fc8afd-4063-47e2-8678-0d4e52b5fd61,"{ ""uploadedBy"": { ""name"": ""Fortuna Zhang"", ""id"": ""c8345eb7-e19e-4d32-ac91-52f61fcf1f8d"" }, ""uploadedOn"": ""2021-11-05T23:24:55.0820000Z"", ""uploadId"": ""6185bd47d08c84086a4440b3"", ""filename"": ""ARPIDFY21Clean2.csv"" }"
0200209597154736823INV,020,36823,95971547,112771753,INV,928.3,GLC-FE-100FX=,100BASE-FX SFP for FE port,020,577fff71-db1a-4792-960c-82eeef04c522,"{ ""uploadedBy"": { ""name"": ""Fortuna Zhang"", ""id"": ""c8345eb7-e19e-4d32-ac91-52f61fcf1f8d"" }, ""uploadedOn"": ""2021-11-05T23:24:55.0820000Z"", ""uploadId"": ""6185bd47d08c84086a4440b3"", ""filename"": ""ARPIDFY21Clean2.csv"" }"
0200209597154836823INV,020,36823,95971548,112761135,INV,1299.6,NIM-24A=,24 Channel Async serial interface for ISR4000 series router,020,4829eed0-b866-48b7-9d2c-48abe22a8083,"{ ""uploadedBy"": { ""name"": ""Fortuna Zhang"", ""id"": ""c8345eb7-e19e-4d32-ac91-52f61fcf1f8d"" }, ""uploadedOn"": ""2021-11-05T23:24:55.0820000Z"", ""uploadId"": ""6185bd47d08c84086a4440b3"", ""filename"": ""ARPIDFY21Clean2.csv"" }"
0200209597154936823INV,020,36823,95971549,112644969,INV,43638.0,CON-SSC4P-SR4X51-X,Cisco ISR 4451 AX Bundle with APP and SEC license,020,fd439578-2365-4a09-90b6-ca94c45e34cf,"{ ""uploadedBy"": { ""name"": ""Fortuna Zhang"", ""id"": "

In [0]:
# Clean up Product ID raw data
PID_raw = PID_raw.select([F.col(col).alias(col.replace('-', '')) for col in PID_raw.columns])
PID_raw = PID_raw.select([F.col(col).alias(col.replace(' ', '_')) for col in PID_raw.columns])

# Add Key column used to join with other dataset
PID_raw = PID_raw.withColumn('PRKey1', trim(concat(col('EntityNumberAccount'), col('EntityNumberSource'), col('Customer_number'), col('TRX_NUMBER'), col('Transaction_Type')))) # Change column names here for customization 
PID_raw.show()

+--------------------+------------------+---------------+----------+-----------+----------------+---------------+------------------+--------------------+-------------------+--------------------+--------------------+--------------------+
 ENTTRXCUSTTYPE|EntityNumberSource|Customer_number|TRX_NUMBER|Sales_Order|Transaction_Type|UnitPriceforPID| PID| PIDDescription|EntityNumberAccount| iris_id| iris_metadata| PRKey1|
+--------------------+------------------+---------------+----------+-----------+----------------+---------------+------------------+--------------------+-------------------+--------------------+--------------------+--------------------+
02002095971544368...| 020| 36823| 95971544| 112697039| INV| 1354.74| PWR-C1-715WAC-P/2|715W AC 80+ plati...| 020|46f03d75-492c-457...|{
 "uploadedBy"...|02002036823959715...|
02002095971544368...| 020| 36823| 95971544| 112697039| INV| 108.36| STACK-T1-50CM|50CM Type 1 Stack...| 020|c354a3c2-7fc5-41f...|{
 "uploadedBy"...|02002036823959715...|
02002095971544368...| 020| 36823| 95971544| 112697039| INV| 2763.66| C9300-NM-8X|Catalyst 9300 8 x...| 020|55211e52-1103-4ff...|{
 "uploadedBy"...|02002036823959715...|
02002095971544368...| 020| 36823| 95971544| 112697039| INV| 10841.28| C9300-48P-E|Catalyst 9300 48-...| 020|d2099d86-7b78-4e5...|{
 "uploadedBy"...|02002036823959715...|
02002095971545368...| 020| 36823| 95971545| 112802917| INV| 719.3| C9300-DNA-E-48-3Y|C9300 DNA Essenti...| 020|c1d92e2b-e602-479...|{
 "uploadedBy"...|02002036823959715...|
02002095971545368...| 020| 36823| 95971545| 112802917| INV| 719.3| C9300-DNA-E-48-3Y|C9300 DNA Essenti...| 020|b151bfc4-98c5-441...|{
 "uploadedBy"...|02002036823959715...|
02002095971546368...| 020| 36823| 95971546| 112752910| INV| 2595.4| UCSC-PCIE-QD16GF=|Qlogic QLE2692 du...| 020|e6fc8afd-4063-47e...|{
 "uploadedBy"...|02002036823959715...|
02002095971547368...| 020| 36823| 95971547| 112771753| INV| 928.3| GLC-FE-100FX=|100BASE-FX SFP f...| 020|577fff71-db1a-479...|{
 "uploadedBy"...|02002036823959715...|
02002095971548368...| 020| 36823| 95971548| 112761135| INV| 1299.6| NIM-24A=|24 Channel Async ...| 020|4829eed0-b866-48b...|{
 "uploadedBy"...|02002036823959715...|
02002095971549368...| 020| 36823| 95971549| 112644969| INV| 43638.0|CON-SSC4P-SR4X51-X|Cisco ISR 4451 AX...| 020|fd439578-2365-4a0...|{
 "uploadedBy"...|02002036823959715...|
02002095971549368...| 020| 36823| 95971549| 112644969| INV| 630.0| CON-ECMUS-WS2500|SOLN SUPP SWSS WA...| 020|8a87b2ba-599c-4aa...|{
 "uploadedBy"...|02002036823959715...|
02002095971550368...| 020| 36823| 95971550| 112790608| INV| 9232.2| LIC-MX250-SEC-3YR|Meraki MX250 Adva...| 020|914b9460-7815-431...|{
 "uploadedBy"...|02002036823959715...|
02002095971551368...| 020| 36823| 95971551| 112798885| INV| 11590.44| FLSA1-1X-10-20G=|10G to 20Gbps upg...| 020|57797990-b2df-4cf...|{
 "uploadedBy"...|02002036823959715...|
02002095971552368...| 020| 36823| 95971552| 112793155| INV| 2786.64| Z3-HW|Meraki Z3 Cloud M...| 020|1725d046-95e1-422...|{
 "uploadedBy"...|02002036823959715...|
02002095971553368...| 020| 36823| 95971553| 112739776| INV| 1161.15| MX84-HW|Meraki MX84 Route...| 020|4060b418-11c2-4bf...|{
 "uploadedBy"...|02002036823959715...|
02002095971554368...| 020| 36823| 95971554| 112247471| INV| 2859.99| N35-F-X16P|Nexus 3550-F Fusi...| 020|6c6b7687-771f-457...|{
 "uploadedBy"...|02002036823959715...|
02002095971554368...| 020| 36823| 95971554| 112247471| INV| 8579.93| N35-FHPT-48X|Nexus 3550-F Fusi...| 020|eaf05d4b-c317-45c...|{
 "uploadedBy"...|02002036823959715...|
02002095971555368...| 020| 36823| 95971555| 112800362| INV| 666.43|LIC-MS250-48FP-3YR|Meraki MS250-48FP...| 020|24786596-65c6-472...|{
 "uploadedBy"...|02002036823959715...|
02002095971556368...| 020| 36823| 95971556| 112247471| INV| 8769.6| CON-SNT-N35FH48X|SNTC-8X5XNBD Nexu...| 020|f77ae4d0-dc20-4fd...|{
 "uploadedBy"...|02002036823959715...|
02002095971557368...| 020| 36823| 95971557| 112724819| INV| 23006.5| L-FPR2120T-

In [0]:
# Create input parameter for key column
dbutils.widgets.dropdown("PID Raw Key Level 1",PID_raw.columns[0], PID_raw.columns)

# Display user selection
dbutils.widgets.get("PID Raw Key Level 1")

Out[12]: 'PRKey1'

In [0]:
# Create temp SQL table
PID_raw.createOrReplaceTempView("PIDRaw")
spark.sql("SELECT * FROM PIDRaw LIMIT 1").toPandas()

Out[10]:

,ENTTRXCUSTTYPE,EntityNumberSource,Customer_number,TRX_NUMBER,Sales_Order,Transaction_Type,UnitPriceforPID,PID,PIDDescription,EntityNumberAccount,iris_id,iris_metadata,PRKey1
0,020020959891091039862866INV,020,1039862866,95989109,112825372,INV,7620.24,CON-PSRT-1,PRTNR SS 8X5XNBD Maintenance,020,7444930c-76fa-451c-bf87-15684090665a,"{\r\n ""uploadedBy"": {\r\n ""name"": ""Fortuna...",020020103986286695989109INV


In [0]:

CR_df = spark.read.format("csv") \
        .option("header", "true") \
        .option("delimiter", ",") \
        .option("inferSchema","true") \
        .option("quote", "\"") \
        .option("escape", "\"") \
        .option("encoding", "UTF-8")\
        .load('dbfs:/mnt/sources/616a0f6d323eaf3d7cdff208/616a0f85323eafb460dff209/Currency Rate Exchange.csv')

CR_df.show()

+---+-------------+--------+--------+--------------+
 LE| Country|Currency| Period |Fx rate to USD|
+---+-------------+--------+--------+--------------+
 20|United States| USD|P12 2020| 1.0|
 20|United States| USD|P12 2019| 1.0|
 20|United States| USD| P3 2021| 1.0|
 20|United States| USD| P6 2021| 1.0|
 20|United States| USD| P9 2021| 1.0|
 20|United States| USD|P12 2021| 1.0|
105| Canada| CAD|P12 2020| 1.34265|
105| Canada| CAD|P12 2019| 1.31345|
105| Canada| CAD| P3 2021| 1.31195|
105| Canada| CAD| P6 2021| 1.27015|
105| Canada| CAD| P9 2021| 1.23965|
105| Canada| CAD|P12 2021| 1.25855|
115| Australia| AUD|P12 2020| 1.39441|
115| Australia| AUD|P12 2019| 1.43184|
115| Australia| AUD| P3 2021| 1.40795|
115| Australia| AUD| P6 2021| 1.29408|
115| Australia| AUD| P9 2021| 1.29074|
115| Australia| AUD|P12 2021| 1.36361|
130| Korea| KRW| P3 2021| 1131.65|
130| Korea| KRW| P6 2021| 1100.2|
+---+-------------+--------+--------+--------------+
only showing top 20 rows

In [0]:
# PID_raw_USD = spark.sql("SELECT *, \
#                                 UnitPriceforPID * 'Fx rate to USD' AS UP_USD \
#                          FROM PIDRaw \
#                          WHERE Period == 'P12 2021'")

# PID_raw_USD.createOrReplaceTempView("PIDRawUSD")
# PID_raw_USD.show()

In [0]:
## Read Product ID level and Invoice level data from Blob storage
## csv files are easier to ingest on CE than Excel

# Product ID data
PID_df = spark.read.format("csv") \
        .option("header", "true") \
        .option("delimiter", ",") \
        .option("inferSchema","true") \
        .option("quote", "\"") \
        .option("escape", "\"") \
        .option("encoding", "UTF-8")\
        .load('dbfs:/mnt/sources/616d9623323eaf5169dff20c/616d964d323eaf5c43dff20d/AR PID Data - Output Total FY21 v2.csv')

# Invoice data (Text files from clients) 
TXT_df = spark.read.format("csv") \
        .option("header", "true") \
        .option("delimiter", ",") \
        .option("inferSchema","true") \
        .option("quote", "\"") \
        .option("escape", "\"") \
        .option("encoding", "UTF-8")\
        .load('dbfs:/mnt/sources/616a021a323eaf3912dff206/616a022f323eaf3150dff207/AR PID Data - TXT FY21 v2.csv')


In [0]:
# Check column type
PID_df.dtypes

Out[53]: [('ENT-TRX-CUST-TYPE', 'string'),
 ('Entity Number - Source', 'int'),
 ('Entity Number - Account', 'int'),
 ('Entity Number - Match', 'int'),
 ('Account', 'string'),
 ('TRX_NUMBER', 'string'),
 ('Sales_Order', 'double'),
 ('Customer_number', 'bigint'),
 ('CUSTOMER_ID', 'int'),
 ('CUSTOMER_TRX_ID', 'int'),
 ('Transaction_Type', 'string'),
 ('GL_DATE_INV', 'string'),
 ('DUE_DATE', 'string'),
 ('DUE_DATE_YEAR', 'int'),
 ('Date Category', 'string'),
 ('INVOICE_CURRENCY_CODE', 'string'),
 ('Sum_AMOUNT_DUE_ORGINAL_USD', 'double'),
 ('Sum_AMOUNT_DUE_ORGINAL', 'double'),
 ('Sum_AMOUNT_DUE_REMAINING', 'double'),
 ('Sum_AMOUNT_LINE_ITEMS_ORIGINAL', 'double'),
 ('Sum_FREIGHT_ORIGINAL', 'double'),
 ('Sum_TAX_ORIGINAL', 'double'),
 ('Sum_AMOUNT_ADJUSTED', 'double'),
 ('Sum_AMOUNT_APPLIED', 'double'),
 ('Sum_AMOUNT_CREDITED', 'double'),
 ('Sum_AMOUNT_ADJUSTED_PENDING', 'double'),
 ('Sum_UNITS', 'double'),
 ('Sum_Unit Price for PID', 'double'),
 ('Sum_UNIT_SELLING_PRICE', 'double'),
 ('Sum_ UNIT_SELLING_PRICE_USD', 'int'),
 ('FileName', 'string')]

In [0]:
# Check column type
TXT_df.dtypes

Out[54]: [('ENT-INV-CUST-TYPE', 'string'),
 ('Company Segment', 'int'),
 ('Reporting Entity', 'string'),
 ('Accounting Flexfield', 'string'),
 ('Invoice Number', 'string'),
 ('Cust Name', 'string'),
 ('Cust Number', 'bigint'),
 ('Type', 'string'),
 ('Type1', 'string'),
 ('Due date', 'string'),
 ('Date Category', 'string'),
 ('Bucket', 'string'),
 ('Outstanding amount', 'double'),
 ('Current', 'double'),
 ('1-30 Days Past due', 'double'),
 ('31-60 days Past due', 'double'),
 ('61-90 days past due', 'double'),
 ('91-180 Days past due', 'double'),
 ('181-360 days past due', 'double'),
 ('361+ days past due', 'double'),
 ('FileName', 'string')]

In [0]:
# Clean up PID and TXT dataframes
PID_df = PID_df.withColumnRenamed("ENT-TRX-CUST-TYPE", "ENTTRXCUSTTYPE")
TXT_df = TXT_df.withColumnRenamed("ENT-INV-CUST-TYPE", "ENTINVCUSTTYPE")
PID_df = PID_df.select([F.col(col).alias(col.replace('-', '')) for col in PID_df.columns])
TXT_df = TXT_df.select([F.col(col).alias(col.replace('-', '')) for col in TXT_df.columns])
PID_df = PID_df.select([F.col(col).alias(col.replace(' ', '_')) for col in PID_df.columns])
TXT_df = TXT_df.select([F.col(col).alias(col.replace(' ', '_')) for col in TXT_df.columns])
PID_df = PID_df.withColumn("Entity_Number__Source", when(PID_df.Entity_Number__Source == "20","020") \
                                                     .otherwise(PID_df.Entity_Number__Source)) \
               .withColumn("Entity_Number__Account", when(PID_df.Entity_Number__Account == "20","020") \
                                                     .otherwise(PID_df.Entity_Number__Account)) 
TXT_df = TXT_df.withColumn("Company_Segment", when(TXT_df.Company_Segment == "20","020") \
                                              .otherwise(TXT_df.Company_Segment))

In [0]:
## Create dynamic key column function 

# If user selects Level 1, will only create Key1; if user selects Level 2, will create Key1 and Key2; and so on. 
def Key_Creation(Level1, Level2, Level3, df1, df2):
  def Key1(df1,df2):
    df1 = df1.withColumn('PKey1', trim(concat(col('Entity_Number__Account'), col('Entity_Number__Source'), col('Customer_number'), col('TRX_NUMBER'), col('Transaction_Type'))))
    df2 = df2.withColumn('TKey1', trim(concat(col('Company_Segment'), col('Company_Segment'), col('Cust_Number'), col('Invoice_Number'), col('Type1'))))
    return df1, df2
  
  def Key2(df1,df2):
    df1 = df1.withColumn('PKey2', trim(col('Customer_number')))
    df2 = df2.withColumn('TKey2', trim(col('Cust_Number')))
    return df1, df2

  def Key3(df1,df2):
    df1 = df1.withColumn('PKey3', trim(substring('Account',1,3)))
    df2 = df2.withColumn('TKey3', trim(col('Company_Segment')))
    return df1, df2
  
  # Create key base on level selection
  if getArgument('Allocation Level') == Level1:
    df1, df2 = Key1(df1, df2)
  elif getArgument('Allocation Level') == Level2:
    df1, df2 = Key1(df1, df2)
    df1, df2 = Key2(df1, df2)
  elif getArgument('Allocation Level') == Level3:
    df1, df2 = Key1(df1, df2)
    df1, df2 = Key2(df1, df2)
    df1, df2 = Key3(df1, df2)
    
  return df1, df2


# Display result dynamically per user selection
Key_Creation(Level1, Level2, Level3, PID_df, TXT_df)

Out[15]: (+--------------------+---------------------+----------------------+--------------------+--------------------+----------+-----------+---------------+-----------+---------------+----------------+-----------+----------+-------------+-------------+---------------------+--------------------------+----------------------+------------------------+------------------------------+--------------------+----------------+-------------------+------------------+-------------------+---------------------------+---------+----------------------+----------------------+---------------------------+--------------------+--------------------+---------+-----+
 ENTTRXCUSTTYPE|Entity_Number__Source|Entity_Number__Account|Entity_Number__Match| Account|TRX_NUMBER|Sales_Order|Customer_number|CUSTOMER_ID|CUSTOMER_TRX_ID|Transaction_Type|GL_DATE_INV| DUE_DATE|DUE_DATE_YEAR|Date_Category|INVOICE_CURRENCY_CODE|Sum_AMOUNT_DUE_ORGINAL_USD|Sum_AMOUNT_DUE_ORGINAL|Sum_AMOUNT_DUE_REMAINING|Sum_AMOUNT_LINE_ITEMS_ORIGINAL|Sum_FREIGHT_ORIGINAL|Sum_TAX_ORIGINAL|Sum_AMOUNT_ADJUSTED|Sum_AMOUNT_APPLIED|Sum_AMOUNT_CREDITED|Sum_AMOUNT_ADJUSTED_PENDING|Sum_UNITS|Sum_Unit_Price_for_PID|Sum_UNIT_SELLING_PRICE|Sum__UNIT_SELLING_PRICE_USD| FileName| PKey1| PKey2|PKey3|
 +--------------------+---------------------+----------------------+--------------------+--------------------+----------+-----------+---------------+-----------+---------------+----------------+-----------+----------+-------------+-------------+---------------------+--------------------------+----------------------+------------------------+------------------------------+--------------------+----------------+-------------------+------------------+-------------------+---------------------------+---------+----------------------+----------------------+---------------------------+--------------------+--------------------+---------+-----+
02002000001378360...| 020| 020| -1|020-000-000000-11...| 000013783| null| 6095| 6095| 205241394| DM| 2004-05-24|2004-06-23| 2004| Pre 2018| USD| 3924.0| 3924.16| 0.0| 3924.16| 0.0| 0.0| -30.06| 3894.1| 0.0| null| 3924.16| 3924.16| 1.0| 1|P12 2021 FYE file...|02002060950000137...| 6095| 020|
02002000001558560...| 020| 020| -1|020-000-000000-11...| 000015585| null| 6095| 6095| 206265125| DM| 2005-04-20|2005-05-20| 2005| Pre 2018| USD| 206.0| 205.91| 0.0| 205.91| 0.0| 0.0| null| 205.91| null| null| 205.91| 205.91| 1.0| 1|P12 2021 FYE file...|02002060950000155...| 6095| 020|
02002000001805450...| 020| 020| -1|020-000-000000-11...| 000018054| null| 509155| 509195| 190277833| DM| 2006-08-17|2006-09-16| 2006| Pre 2018| USD| 14419.0| 14418.99| 0.0| 14418.99| 0.0| 0.0| null| 14418.99| null| null| 14418.99| 14418.99| 1.0| 1|P12 2021 FYE file...|02002050915500001...| 509155| 020|
02002000001892510...| 020| 020| -1|020-000-000000-11...| 000018925| null| 1061806| 1062197| 208872771| DM| 2007-04-17|2007-05-17| 2007| Pre 2018| USD| 634.0| 634.17| 0.0| 634.17| 0.0| 0.0| null| 634.17| null| null| 634.17| 634.17| 1.0| 1|P12 2021 FYE file...|02002010618060000...| 1061806| 020|
02002000001958640...| 020| 020| -1|020-000-000000-11...| 000019586| null| 400172765| 400188162| 209487756| DM| 2007-09-17|2007-10-17| 2007| Pre 2018| USD| 3728.0| 3728.32| 0.0| 3728.32| 0.0| 0.0| null| 3728.32| null| null| 3728.32| 3728.32| 1.0| 1|P12 2021 FYE file...|02002040017276500...|400172765| 020|
02002000001958850...| 020| 020| -1|020-000-000000-11...| 000019588| null| 507851| 507891| 209605776| DM| 2007-10-16|2007-11-15| 2007| Pre 2018| USD| 250.0| 250.37| 0.0| 250.37| 0.0| 0.0| null| 250.37| null| null| 250.37| 250.37| 1.0| 1|TXT Only Invoice ...|02002050785100001...| 507851| 020|
02002000001958850...| 020| 020| -1|020-000-000000-11...| 000019588| null| 507851| 507891| 209605776| DM| 2007-10-16|2007-11-15| 2007| Pre 2018| USD| 250.0| 250.37| 0.0| 250.37| 0.0| 0.0| null| 250.37| null| null| 500.74| 500.74| 2.0| 2|P12 2021 FYE file...|02002050785100001...| 507851| 020|
02002000001962760...| 020| 020| -1|020-000-000000

In [0]:
# Assign key columns to dataframes
PID_df, TXT_df = Key_Creation(Level1, Level2, Level3, PID_df, TXT_df)

In [0]:
# Create input parameter
dbutils.widgets.dropdown("PID Key Level 1",PID_df.columns[0], PID_df.columns)

# Display user selection
dbutils.widgets.get("PID Key Level 1")

Out[17]: 'PKey1'

In [0]:
# Create input parameter
dbutils.widgets.dropdown("TXT Key Level 1",TXT_df.columns[0], TXT_df.columns)

# Display user selection
dbutils.widgets.get("TXT Key Level 1")

Out[18]: 'TKey1'

In [0]:
# Create SQL temp table 
PID_df.createOrReplaceTempView("PID")
spark.sql("SELECT * FROM PID LIMIT 1").toPandas()

Out[19]:

,ENTTRXCUSTTYPE,Entity_Number__Source,Entity_Number__Account,Entity_Number__Match,Account,TRX_NUMBER,Sales_Order,Customer_number,CUSTOMER_ID,CUSTOMER_TRX_ID,Transaction_Type,GL_DATE_INV,DUE_DATE,DUE_DATE_YEAR,Date_Category,INVOICE_CURRENCY_CODE,Sum_AMOUNT_DUE_ORGINAL_USD,Sum_AMOUNT_DUE_ORGINAL,Sum_AMOUNT_DUE_REMAINING,Sum_AMOUNT_LINE_ITEMS_ORIGINAL,Sum_FREIGHT_ORIGINAL,Sum_TAX_ORIGINAL,Sum_AMOUNT_ADJUSTED,Sum_AMOUNT_APPLIED,Sum_AMOUNT_CREDITED,Sum_AMOUNT_ADJUSTED_PENDING,Sum_UNITS,Sum_Unit_Price_for_PID,Sum_UNIT_SELLING_PRICE,Sum__UNIT_SELLING_PRICE_USD,FileName,PKey1,PKey2,PKey3
0,02002061008356351014699899INV,020,020,-1,020-000-000000-11501-000-000000,6100835635,2072708.0,1014699899,1014760305,629637418,INV,2021-03-01,2021-04-30,2021,Post 2018,USD,89.0,88.7,88.7,88.7,0.0,0.0,NaN,NaN,NaN,NaN,5.0,88.7,17.74,18,P12 2021 FYE files for Q4FY21 PID - 3,02002010146998996100835635INV,1014699899,020


In [0]:
# Create SQL temp table 
TXT_df.createOrReplaceTempView("TXT")
spark.sql("SELECT * FROM TXT LIMIT 1").toPandas()

Out[21]:

,ENTINVCUSTTYPE,Company_Segment,Reporting_Entity,Accounting_Flexfield,Invoice_Number,Cust_Name,Cust_Number,Type,Type1,Due_date,Date_Category,Bucket,Outstanding_amount,Current,130_Days_Past_due,3160_days_Past_due,6190_days_past_due,91180_Days_past_due,181360_days_past_due,361+_days_past_due,FileName,TKey1,TKey2,TKey3
0,1931931559271000023425INV,193,Cisco Brazil CA,193-000-000000-11501-000-000000,155927,WESTCON BRASILLTDA,1000023425,INV-STD-SVC-SP,INV,23-AUG-21,Post 2018,Current,1651.34,1651.34,NaN,NaN,NaN,NaN,NaN,NaN,Aging___7_Buckets____By_Accoun_010821_BRAZIL_INV,1931931000023425155927INV,1000023425,193


In [0]:
#### 3 scenarios for PID and TXT data joining by level 1 key ####
## 1) Level1 key exists in both text and PID files
## 2) Level1 key only exists in PID file
## 3) Level1 key only exists in text file
## AR allocation model will take case 1 and case 3 into account, i.e. to include all outstanding amount from text/invoice level data. Case 1 will be allocated at Key1 level, while case 3 will be allocated at higher key level. 

## 1) Both exist in text and PID files ####
# Get key column to join from user input 
PID_key1 = getArgument('PID Key Level 1')
TXT_key1 = getArgument('TXT Key Level 1')

## Combine PID and txt/invoice data at key1 level, to compare unit price and outstanding amount
PID_agg = spark.sql("SELECT DISTINCT {}, SUM(Sum_AMOUNT_DUE_REMAINING) AS ADR, SUM(Sum_Unit_Price_for_PID) AS UP, SUM(Sum_TAX_ORIGINAL) AS Tax \
                     FROM PID \
                     GROUP BY {}".format(PID_key1,PID_key1))
TXT_agg = spark.sql("SELECT DISTINCT {}, SUM(Outstanding_amount) AS OA \
                     FROM TXT \
                     GROUP BY {}".format(TXT_key1,TXT_key1))

# Create temp SQL table
PID_agg.createOrReplaceTempView("PIDAgg")
TXT_agg.createOrReplaceTempView("TXTAgg")

# Recon - tie out amounts from two sources
Inner_Join_df = spark.sql("SELECT * \
                          ,'In PID and TXT' AS Tie_to_PID_and_TXT \
                          ,CASE WHEN ROUND(ADR) = 0 AND ROUND(OA) = 0 THEN 1 \
                                WHEN ADR/OA > 0.9 AND ADR/OA < 1.1 THEN 1 \
                                ELSE 0 \
                                END AS Tie_to_Amount_Due_Remaining \
                          ,CASE WHEN ROUND(UP + Tax) = 0 AND ROUND(OA) = 0 THEN 1 \
                                WHEN (UP + Tax)/OA > 0.9 AND (UP + Tax)/OA < 1.1 THEN 1 \
                                ELSE 0 \
                                END AS Tie_to_Unit_Price_Plus_Tax \
                          ,CASE WHEN ROUND(UP) = 0 AND ROUND(OA) = 0 THEN 1 \
                                WHEN UP/OA > 0.9 AND UP/OA < 1.1 THEN 1 \
                                ELSE 0 \
                                END AS Tie_to_Unit_Price \
                           FROM PIDAgg \
                           INNER JOIN TXTAgg \
                           ON PIDAgg.{} == TXTAgg.{}".format(PID_key1, TXT_key1))

Inner_Join_df.createOrReplaceTempView("Inner")
Inner_Join_df.show()

+--------------------+--------+--------+------+--------------------+--------+------------------+---------------------------+--------------------------+-----------------+
 PKey1| ADR| UP| Tax| TKey1| OA|Tie_to_PID_and_TXT|Tie_to_Amount_Due_Remaining|Tie_to_Unit_Price_Plus_Tax|Tie_to_Unit_Price|
+--------------------+--------+--------+------+--------------------+--------+------------------+---------------------------+--------------------------+-----------------+
02002013486610085...| 11328.0| 11328.0| 0.0|02002013486610085...| 11328.0| In PID and TXT| 1| 1| 1|
02002040174781161...| 20.26| 20.26| 0.0|02002040174781161...| 20.26| In PID and TXT| 1| 1| 1|
02002010357630996...| 101.7| 2034.0| 101.7|02002010357630996...| 101.7| In PID and TXT| 1| 0| 0|
02002010291026100...| 309.9| 7334.88| 309.9|02002010291026100...| 309.9| In PID and TXT| 1| 0| 0|
02002024466100863...| 414.7| 414.7| 0.0|02002024466100863...| 414.7| In PID and TXT| 1| 1| 1|
02002010380454446...|56530.44|56530.44| 0.0|02002010380454446...| 8479.57| In PID and TXT| 0| 0| 0|
02002040022310761...| 71.7| 71.7| 0.0|02002040022310761...| 71.7| In PID and TXT| 1| 1| 1|
02002090776100866...| 0.0| 4410.0| 0.0|02002090776100866...| 4410.0| In PID and TXT| 0| 1| 1|
02002040547647861...| 0.0| -60.05| 0.0|02002040547647861...| -0.76| In PID and TXT| 0| 0| 0|
02002084829610087...| 2.99| 35.48| 2.99|02002084829610087...| 2.99| In PID and TXT| 1| 0| 0|
02002040022310761...| 165.36| 165.36| 0.0|02002040022310761...| 165.36| In PID and TXT| 1| 1| 1|
02002013486610088...| 2799.48| 2799.48| 0.0|02002013486610088...| 2799.48| In PID and TXT| 1| 1| 1|
02002040174781061...| 0.0| -18.14| 0.0|02002040174781061...| -18.14| In PID and TXT| 0| 1| 1|
02002040751310161...| 0.0| 2880.0| 0.0|02002040751310161...| 2880.0| In PID and TXT| 0| 1| 1|
02002010357736100...| 135.49| 2709.74|135.49|02002010357736100...| 135.49| In PID and TXT| 1| 0| 0|
02002010357630996...| 8.0| 78.0| 8.0|02002010357630996...| 8.0| In PID and TXT| 1| 0| 0|
02002013706100906...| 0.0| 6408.7| 0.0|02002013706100906...| 4184.44| In PID and TXT| 0| 0| 0|
02002024466100906...|12776.14|12052.96|723.18|02002024466100906...|12776.14| In PID and TXT| 1| 1| 1|
02002013706100906...| 0.0| 3733.58| 0.0|02002013706100906...| 3733.58| In PID and TXT| 0| 1| 1|
02002013706100906...| 0.0| 85738.0| 0.0|02002013706100906...| 85738.0| In PID and TXT| 0| 1| 1|
+--------------------+--------+--------+------+--------------------+--------+------------------+---------------------------+--------------------------+-----------------+
only showing top 20 rows

In [0]:
## Get entire datasets from two sources for key1 that exist in both TXT and PID data
PID_agg2 = spark.sql("SELECT DISTINCT {} \
                            ,NULL AS Reporting_Entity \
                            ,Entity_Number__Source AS Company_Segment \
                            ,TRX_NUMBER AS Invoice_Number\
                            ,Customer_number \
                            ,Transaction_Type \
                            ,NULL AS Bucket \
                            ,FileName \
                            ,SUM(Sum_AMOUNT_DUE_REMAINING) AS ADR \
                            ,SUM(Sum_Unit_Price_for_PID) AS UP \
                            ,SUM(Sum_TAX_ORIGINAL) AS Tax \
                            ,NULL AS OA \
                      FROM PID \
                      GROUP BY {}, Reporting_Entity, Company_Segment, Invoice_Number, Customer_number, Transaction_Type, Bucket, FileName".format(PID_key1,PID_key1))

TXT_agg2 = spark.sql("SELECT DISTINCT {} \
                            ,Reporting_Entity \
                            ,Company_Segment \
                            ,Invoice_Number \
                            ,Cust_Number AS Customer_number\
                            ,Type1 AS Transaction_Type\
                            ,Bucket \
                            ,FileName \
                            ,NULL AS ADR \
                            ,NULL AS UP \
                            ,NULL AS Tax \
                            ,SUM(Outstanding_amount) AS OA \
                      FROM TXT \
                      GROUP BY {}, Reporting_Entity, Company_Segment, Invoice_Number, Customer_number, Transaction_Type, Bucket, FileName".format(TXT_key1,TXT_key1))

# Create temp SQL table
PID_agg2.createOrReplaceTempView("PIDAgg2")
TXT_agg2.createOrReplaceTempView("TXTAgg2")

# Combine PID and txt/invoice data with matched key 1 value
Inner_Join_df2 = spark.sql("SELECT i.{} AS Key1 \
                                  ,u.Reporting_Entity \
                                  ,u.Company_Segment \
                                  ,u.Invoice_Number \
                                  ,u.Customer_number \
                                  ,u.Transaction_Type \
                                  ,u.Bucket \
                                  ,u.FileName \
                                  ,u.OA AS Outstanding_Amount \
                                  ,u.UP \
                                  ,Tie_to_PID_and_TXT \
                                  ,Tie_to_Amount_Due_Remaining \
                                  ,Tie_to_Unit_Price_Plus_Tax \
                                  ,Tie_to_Unit_Price \
                            FROM Inner AS i\
                            INNER JOIN \
                               (SELECT *, {} AS Key1 \
                                FROM PIDAgg2 \
                                UNION \
                                SELECT *, {} AS Key1 \
                                FROM TXTAgg2) AS u \
                            ON i.{} == u.Key1".format(TXT_key1, PID_key1, TXT_key1, TXT_key1))

# Create temp SQL table
Inner_Join_df2.createOrReplaceTempView("Inner2")
Inner_Join_df2.show()

+--------------------+----------------+---------------+--------------+---------------+----------------+--------------------+--------------------+------------------+--------+------------------+---------------------------+--------------------------+-----------------+
 Key1|Reporting_Entity|Company_Segment|Invoice_Number|Customer_number|Transaction_Type| Bucket| FileName|Outstanding_Amount| UP|Tie_to_PID_and_TXT|Tie_to_Amount_Due_Remaining|Tie_to_Unit_Price_Plus_Tax|Tie_to_Unit_Price|
+--------------------+----------------+---------------+--------------+---------------+----------------+--------------------+--------------------+------------------+--------+------------------+---------------------------+--------------------------+-----------------+
02002010000414696...| null| 020| 6100943780| 1000041469| INV| null|P12 2021 FYE file...| null| 225.6| In PID and TXT| 0| 1| 1|
02002010000414696...| Cisco US| 020| 6100943780| 1000041469| INV| Current|Aging___7_Buckets...| 225.6| null| In PID and TXT| 0| 1| 1|
02002010000813149...| Cisco US| 020| 95416437| 1000081314| INV|91-180 Days Past due|Aging___7_Buckets...| 3236.8| null| In PID and TXT| 1| 1| 1|
02002010000813149...| null| 020| 95416437| 1000081314| INV| null|P12 2021 FYE file...| null| 3236.8| In PID and TXT| 1| 1| 1|
02002010000954678...| Cisco US| 020| 80393823| 1000095467| CM| 1-30 Days Past due|Aging___7_Buckets...| -26.09| null| In PID and TXT| 0| 1| 1|
02002010000954678...| null| 020| 80393823| 1000095467| CM| null|P12 2021 FYE file...| null| -26.09| In PID and TXT| 0| 1| 1|
02002010000954679...| Cisco US| 020| 95981137| 1000095467| INV| Current|Aging___7_Buckets...| 349.22| null| In PID and TXT| 0| 1| 1|
02002010000954679...| null| 020| 95981137| 1000095467| INV| null|P12 2021 FYE file...| null| 349.22| In PID and TXT| 0| 1| 1|
02002010000968819...| null| 020| 95966259| 1000096881| INV| null|P12 2021 FYE file...| null| 3886.0| In PID and TXT| 0| 1| 1|
02002010000968819...| Cisco US| 020| 95966259| 1000096881| INV| Current|Aging___7_Buckets...| 3886.0| null| In PID and TXT| 0| 1| 1|
02002010001038796...| null| 020| 6100770671| 1000103879| INV| null|P12 2021 FYE file...| null| 1162.2| In PID and TXT| 1| 1| 1|
02002010001038796...| Cisco US| 020| 6100770671| 1000103879| INV|181-360 Days Past...|Aging___7_Buckets...| 1162.2| null| In PID and TXT| 1| 1| 1|
02002010001205191...| null| 020| 1000083098| 1000120519| CM| null|P12 2021 FYE file...| null| -87.91| In PID and TXT| 1| 1| 1|
02002010001205191...| Cisco US| 020| 1000083098| 1000120519| CM|91-180 Days Past due|Aging___7_Buckets...| -87.91| null| In PID and TXT| 1| 1| 1|
02002010001455879...| null| 020| 95784312| 1000145587| INV| null|P12 2021 FYE file...| null| 5147.88| In PID and TXT| 1| 1| 1|
02002010001455879...| Cisco US| 020| 95784312| 1000145587| INV| 1-30 Days Past due|Aging___7_Buckets...| 5506.63| null| In PID and TXT| 1| 1| 1|
02002010001476366...| null| 020| 6100938530| 1000147636| INV| null|P12 2021 FYE file...| null| 10143.0| In PID and TXT| 1| 1| 1|
02002010001476366...| Cisco US| 020| 6100938530| 1000147636| INV| Current|Aging___7_Buckets...| 10143.0| null| In PID and TXT| 1| 1| 1|
02002010001525619...| null| 020| 95973562| 1000152561| INV| null|P12 2021 FYE file...| null|54201.52| In PID and TXT| 0| 1| 1|
02002010001525619...| Cisco US| 020| 95973562| 1000152561| INV| Current|Aging___7_Buckets...| 54201.52| null| In PID and TXT| 0| 1| 1|
+--------------------+----------------+---------------+--------------+---------------+----------------+--------------------+--------------------+------------------+--------+------------------+---------------------------+--------------------------+-----------------+
only showing top 20 rows

In [0]:
#### 2) Key1 Only in PID files ####
## This case will not be considered into AR allocation
PID_Only_df = spark.sql("SELECT * \
                          FROM PIDAgg2 \
                          LEFT ANTI JOIN Inner \
                          ON PIDAgg2.{} == Inner.{}".format(PID_key1,PID_key1))

# Create temp SQL table
PID_Only_df.createOrReplaceTempView("PIDOnly")
PID_Only_df.show()

+--------------------+----------------+---------------+--------------+---------------+----------------+------+--------------------+---+--------+-------+----+
 PKey1|Reporting_Entity|Company_Segment|Invoice_Number|Customer_number|Transaction_Type|Bucket| FileName|ADR| UP| Tax| OA|
+--------------------+----------------+---------------+--------------+---------------+----------------+------+--------------------+---+--------+-------+----+
02002040057902598...| null| 020| 9025986| 40057| INV| null|P12 2021 FYE file...|0.0| 32.5| 0.0|null|
02002022513902633...| null| 020| 9026332| 22513| INV| null|P12 2021 FYE file...|0.0| 5197.0| 0.0|null|
02002022513902635...| null| 020| 9026359| 22513| INV| null|P12 2021 FYE file...|0.0| 14.0| 0.0|null|
02002025877902643...| null| 020| 9026437| 25877| INV| null|P12 2021 FYE file...|0.0| 13915.0| 0.0|null|
02002022513902657...| null| 020| 9026578| 22513| INV| null|P12 2021 FYE file...|0.0| 5502.0| 0.0|null|
02002022513902672...| null| 020| 9026720| 22513| INV| null|P12 2021 FYE file...|0.0| 480.0| 0.0|null|
02002022513902687...| null| 020| 9026879| 22513| INV| null|P12 2021 FYE file...|0.0| 299.2| 0.0|null|
02002022513902696...| null| 020| 9026964| 22513| INV| null|P12 2021 FYE file...|0.0| 1536.0| 0.0|null|
02002038998902721...| null| 020| 9027218| 38998| INV| null|P12 2021 FYE file...|0.0| 1710.0| 0.0|null|
02002057820990273...| null| 020| 9027338| 578209| INV| null|P12 2021 FYE file...|0.0| 17906.4| 1477.3|null|
02002036823902741...| null| 020| 9027417| 36823| INV| null|P12 2021 FYE file...|0.0| 34125.0| 0.0|null|
02002074899027812INV| null| 020| 9027812| 7489| INV| null|P12 2021 FYE file...|0.0| 21592.5| 0.0|null|
02002079232902783...| null| 020| 9027834| 79232| INV| null|P12 2021 FYE file...|0.0| 1797.75| 0.0|null|
02002012579028555INV| null| 020| 9028555| 1257| INV| null|TXT Only Invoice ...|0.0| 1218.0| 0.0|null|
02002059341790259...| null| 020| 9025959| 593417| INV| null|P12 2021 FYE file...|0.0| 40050.0| 0.0|null|
02002036823902642...| null| 020| 9026420| 36823| INV| null|P12 2021 FYE file...|0.0| 1170.0| 0.0|null|
02002027012902652...| null| 020| 9026522| 27012| INV| null|P12 2021 FYE file...|0.0| 1740.0| 0.0|null|
02002038996902723...| null| 020| 9027232| 38996| INV| null|P12 2021 FYE file...|0.0| 48565.4| 0.0|null|
02002064119027831INV| null| 020| 9027831| 6411| INV| null|P12 2021 FYE file...|0.0|107522.4| 0.0|null|
02002010180489028...| null| 020| 9028142| 1018048| INV| null|P12 2021 FYE file...|0.0|210000.0|16275.0|null|
+--------------------+----------------+---------------+--------------+---------------+----------------+------+--------------------+---+--------+-------+----+
only showing top 20 rows

In [0]:
#### 3) Key1 Only in TXT files ####
## This case will be considered in AR allocation at higher level
TXT_Only_df = spark.sql("SELECT * \
                          FROM TXTAgg2 \
                          LEFT ANTI JOIN Inner \
                          ON TXTAgg2.{} == Inner.{}".format(TXT_key1,TXT_key1))

TXT_Only_df.createOrReplaceTempView("TXTOnly")
TXT_Only_df.show()

+--------------------+--------------------+---------------+-------------------+---------------+----------------+--------------------+--------------------+----+----+----+----------+
 TKey1| Reporting_Entity|Company_Segment| Invoice_Number|Customer_number|Transaction_Type| Bucket| FileName| ADR| UP| Tax| OA|
+--------------------+--------------------+---------------+-------------------+---------------+----------------+--------------------+--------------------+----+----+----+----------+
02002040485531821...| Cisco US| 020| 2118367053| 404855318| CM| 31-60 Days Past due|Aging___7_Buckets...|null|null|null| -3858.13|
02002036823DM6100...| Cisco US| 020| DM6100908564DMTAX| 36823| DM| Current|Aging___7_Buckets...|null|null|null| 269.61|
02002054441000012...| Cisco US| 020| 000012427| 54441| Payment| 1-30 Days Past due|Aging___7_Buckets...|null|null|null| -2009.98|
69669610127067367...|CISCO RUSSIA SOL ...| 696| 700233551| 1012706736| INV| Current| Russia_P12FY21|null|null|null| 27957.99|
02002036823DM9529...| Cisco US| 020| DM95292377DMTAX| 36823| DM|91-180 Days Past due|Aging___7_Buckets...|null|null|null| 210.58|
62462410272356546...| Cisco UKHome| 624| 6008182D1Q49| 1027235654| Payment| 361+ Days Past due|Aging___7_Buckets...|null|null|null| -11510.0|
02002010401339482...| Cisco US| 020| 2118373737| 1040133948| CM| 1-30 Days Past due|Aging___7_Buckets...|null|null|null| -108.0|
02002036823DM9531...| Cisco US| 020| DM95314414DMTAX| 36823| DM|91-180 Days Past due|Aging___7_Buckets...|null|null|null| 219.27|
02002036823DM9566...| Cisco US| 020| DM95661241DMTAX| 36823| DM| 1-30 Days Past due|Aging___7_Buckets...|null|null|null| 55.87|
02002040179869300...| Cisco US| 020| 00055489| 401798693| Payment|91-180 Days Past due|Aging___7_Buckets...|null|null|null| -11108.65|
69669610127067367...|CISCO RUSSIA SOL ...| 696| 700232369| 1012706736| INV| Current| Russia_P12FY21|null|null|null| 45789.31|
02002010089521430...| Cisco US| 020| 000022056| 1008952143| Payment| 361+ Days Past due|Aging___7_Buckets...|null|null|null| -1274.0|
02002036823DM9580...| Cisco US| 020| DM95809459DMTAX| 36823| DM| Current|Aging___7_Buckets...|null|null|null| 104.77|
115115400338863AU...| Cisco Australia| 115|AUSACH13JAN21106346| 400338863| Payment|181-360 Days Past...|Aging___7_Buckets...|null|null|null| -3990.9|
62462411316738260...| Cisco UKHome| 624| 6008204D9Q30| 113167382| Payment| 1-30 Days Past due|Aging___7_Buckets...|null|null|null|-202294.86|
02002036823DM9579...| Cisco US| 020| DM95791495DMTAX| 36823| DM| Current|Aging___7_Buckets...|null|null|null| 59.6|
02002057190000123...| Cisco US| 020| 000012384| 5719| Payment| 1-30 Days Past due|Aging___7_Buckets...|null|null|null| -22470.0|
02002067151521183...| Cisco US| 020| 2118369865| 671515| CM| 31-60 Days Past due|Aging___7_Buckets...|null|null|null| -1485.52|
02002010140DM4044...| Cisco US| 020| DM40440A| 10140| DM| 361+ Days Past due|Aging___7_Buckets...|null|null|null| 954.1|
02002040174781009...| Cisco US| 020| 09150544| 401747810| Payment|91-180 Days Past due|Aging___7_Buckets...|null|null|null| -4450.8|
+--------------------+--------------------+---------------+-------------------+---------------+----------------+--------------------+--------------------+----+----+----+----------+
only showing top 20 rows

In [0]:
#### Flat file includes in scope percentage for service PIDs, 
#### multiply by service unit price to get in scope amount for later AR allcation percentage calculation ####

# Read Flat File from blob
Flat_df = spark.read.format("csv") \
        .option("header", "true") \
        .option("delimiter", ",") \
        .option("inferSchema","true") \
        .option("quote", "\"") \
        .option("escape", "\"") \
        .option("encoding", "UTF-8")\
        .load('dbfs:/mnt/sources/616def74323eafe485dff210/616def82323eaf8e4ddff211/Cisco_Carmel_TSS_AS_Rev_SO_Percentage.csv')

# Create SQL temp table
Flat_df.createOrReplaceTempView("Flat")
Flat_df.show()

+-------------------+--------+-------------+------------+------------+-----------+--------------+------------+-------------+-----------+--------------+-------------+--------------+-------------+---------------+--------------+------------------+-----------------+------------------+-----------------+
ERPSalesOrderNumber| Deal_ID|20_Carmel_TSS|20_Carmel_AS|20_Cisco_TSS|20_Cisco_AS|21_Carmel_TSS |21_Carmel_AS|21_Cisco_TSS |21_Cisco_AS|20_TSS_Percent|20_AS_Percent|21_TSS_Percent|21_AS_Percent|21Q_TSS_Percent|21Q_AS_Percent|20_TSS_Percent_Tag|20_AS_Percent_Tag|21_TSS_Percent_Tag|21_AS_Percent_Tag|
+-------------------+--------+-------------+------------+------------+-----------+--------------+------------+-------------+-----------+--------------+-------------+--------------+-------------+---------------+--------------+------------------+-----------------+------------------+-----------------+
 108333498|19100775| 0.0| 1943578| 0.0| 6356207.4| 0.0| 0.0| null| 0.0| 0.0| 0.31| 0.0| 0.0| 0.0| 0.31| Normal| Normal| Normal| Normal|
 108464039|18926725| 0.0| 1399229| 0.0| 2741717.72| 0.0| 348534.0| 0.0| 542893.29| 0.0| 0.51| 0.0| 0.64| 0.0| 0.64| Normal| Normal| Normal| Normal|
 108322521|18271899| 0.0| 788414| 0.0| 2166280.29| 0.0| 200341.0| 0.0| 200340.84| 0.0| 0.36| 0.0| 1.0| 0.0| 1.0| Normal| Normal| Normal|Greater than 100%|
 109048624|19894604| 0.0| 762493| 0.0|762493.0775| 0.0| 0.0| null| 0.0| 0.0| 1.0| 0.0| 0.0| 0.0| 1.0| Normal| Normal| Normal| Normal|
 107702865|18855924| 0.0| 756320| 0.0| 756320.4| 0.0| 315134.0| 0.0| 315133.58| 0.0| 1.0| 0.0| 1.0| 0.0| 1.0| Normal| Normal| Normal|Greater than 100%|
 107702867|18855924| 0.0| 378160| 0.0| 378160.32| 0.0| 157567.0| 0.0| 157566.9| 0.0| 1.0| 0.0| 1.0| 0.0| 1.0| Normal| Normal| Normal|Greater than 100%|
 110295040|20786295| 0.0| 354052| 0.0| 354052| 0.0| 0.0| null| 0.0| 0.0| 1.0| 0.0| 0.0| 0.0| 1.0| Normal| Normal| Normal| Normal|
 110474759|20718054| 0.0| 350075| 0.0|350074.7291| 0.0| 414496.0| 0.0|414495.5435| 0.0| 1.0| 0.0| 1.0| 0.0| 1.0| Normal|Greater than 100%| Normal|Greater than 100%|
 109594587|19443836| 0.0| 289960| 0.0| 300000| 0.0| 0.0| null| 0.0| 0.0| 0.97| 0.0| 0.0| 0.0| 0.97| Normal| Normal| Normal| Normal|
 110224891|20615948| 0.0| 251166| 0.0| 251166| 0.0| 0.0| null| 0.0| 0.0| 1.0| 0.0| 0.0| 0.0| 1.0| Normal| Normal| Normal| Normal|
 108989061|19129828| 0.0| 218977| 0.0|218976.7528| 0.0| 0.0| null| 0.0| 0.0| 1.0| 0.0| 0.0| 0.0| 1.0| Normal|Greater than 100%| Normal| Normal|
 109575795|20499222| 0.0| 207608| 0.0| 207608| 0.0| 0.0| null| 0.0| 0.0| 1.0| 0.0| 0.0| 0.0| 1.0| Normal| Normal| Normal| Normal|
 108893095|19405685| 0.0| 186058| 0.0|217802.6603| 0.0| 0.0| null| 0.0| 0.0| 0.85| 0.0| 0.0| 0.0| 0.85| Normal| Normal| Normal| Normal|
 109430247|18877852| 0.0| 185814| 0.0|2974223.058| 0.0| 80348.0| 0.0|1271726.006| 0.0| 0.06| 0.0| 0.06| 0.0| 0.06| Normal| Normal| Normal| Normal|
 108700024|19674508| 0.0| 180000| 0.0| 180000| 0.0| 0.0| null| 0.0| 0.0| 1.0| 0.0| 0.0| 0.0| 1.0| Normal| Normal| Normal| Normal|
 108919102|18691715| 0.0| 170239| 0.0|170239.2646| 0.0| 0.0| 0.0|29930.76408| 0.0| 1.0| 0.0| 0.0| 0.0| 0.0| Normal| Normal| Normal| Normal|
 109896225|18877852| 0.0| 165567| 0.0| 2677501.99| 0.0| 0.0| 0.0|245984.3845| 0.0| 0.06| 0.0| 0.0| 0.0| 0.0| Normal| Normal| Normal| Normal|
 108665475|19501697| 0.0| 158876| 0.0|190514.0505| 0.0| 0.0| null| 0.0| 0.0| 0.83| 0.0| 0.0| 0.0| 0.83| Normal| Normal| Normal| Normal|
 108363044|19469710| 0.0| 153720| 0.0| 177910| 0.0| 0.0| null| 0.0| 0.0| 0.86| 0.0| 0.0| 0.0| 0.86| Normal| Normal| Normal| Normal|
 108893515|19817499| 0.0| 148636| 0.0| 148636.5| 0.0| 0.0| null| 0.0| 0.0| 1.0| 0.0| 0.0| 0.0| 1.0| Normal| Normal| Normal| Normal|
+-------------------+--------+-------------+------------+------------+-----------+--------------+------------+-------------+-----------+--------------+-------------+--------------+-------------+---------------+--------------+------------------+-----------------+------------------+-----------------+

In [0]:
## Get PID granular data for matched key1 set

# Get key column to join from user input 
PR_key1 = getArgument('PID Raw Key Level 1') 

# Inner join
PID_Inner_df = spark.sql("SELECT p.Sales_Order, p.PID, p.PIDDescription, p.UnitPriceforPID, \
                                 i.Key1, i.Company_Segment, i.Customer_number, i.Invoice_Number, i.Transaction_Type, i.Bucket, i.FileName, i.Outstanding_Amount \
                          FROM PIDRaw AS p \
                          INNER JOIN Inner2 AS i \
                          ON p.{} == i.Key1 \
                          WHERE Reporting_Entity IS NOT NULL".format(PR_key1))

PID_Inner_df.show()

+--------------------+-----------------+--------------------+---------------+--------------------+---------------+---------------+--------------+----------------+--------------------+--------------------+------------------+
 Sales_Order| PID| PIDDescription|UnitPriceforPID| Key1|Company_Segment|Customer_number|Invoice_Number|Transaction_Type| Bucket| FileName|Outstanding_Amount|
+--------------------+-----------------+--------------------+---------------+--------------------+---------------+---------------+--------------+----------------+--------------------+--------------------+------------------+
 2487086|DNAC1A-OP-500M-5Y|DNAC1 Advantage O...| 225.6|02002010000414696...| 020| 1000041469| 6100943780| INV| Current|Aging___7_Buckets...| 225.6|
 112078613| L-M9148S-PL12=|MDS 9148S 16G FC ...| 3236.8|02002010000813149...| 020| 1000081314| 95416437| INV|91-180 Days Past due|Aging___7_Buckets...| 3236.8|
 7857903| LIC-MX65-ENT-1YR|EOS Meraki MX65 E...| -26.09|02002010000954678...| 020| 1000095467| 80393823| CM| 1-30 Days Past due|Aging___7_Buckets...| -26.09|
 112743936| LIC-ENT-3YR|Meraki MR Enterpr...| 349.22|02002010000954679...| 020| 1000095467| 95981137| INV| Current|Aging___7_Buckets...| 349.22|
 112800352| R-ISE-VMS-K9=|Cisco ISE Virtual...| 3886.0|02002010000968819...| 020| 1000096881| 95966259| INV| Current|Aging___7_Buckets...| 3886.0|
 1795551| A-FLEX-NU-TPROOM|NU TelePresence R...| 194.7|02002010001038796...| 020| 1000103879| 6100770671| INV|181-360 Days Past...|Aging___7_Buckets...| 1162.2|
 1795551| A-FLEX-NUCM-MC|NU Cloud Meetings...| 967.5|02002010001038796...| 020| 1000103879| 6100770671| INV|181-360 Days Past...|Aging___7_Buckets...| 1162.2|
94346533161831743...| CON-SNT-1|SMARTnet Maintenance| -29.9|02002010001205191...| 020| 1000120519| 1000083098| CM|91-180 Days Past due|Aging___7_Buckets...| -87.91|
94346533161831743...| CON-SNT-1|SMARTnet Maintenance| -58.01|02002010001205191...| 020| 1000120519| 1000083098| CM|91-180 Days Past due|Aging___7_Buckets...| -87.91|
 112535262| FREIGHT| FREIGHT CHARGE| 12.06|02002010001455879...| 020| 1000145587| 95784312| INV| 1-30 Days Past due|Aging___7_Buckets...| 5506.63|
 112535262| XFP-10G-MM-SR=|10GBASE-SR XFP Mo...| 1770.16|02002010001455879...| 020| 1000145587| 95784312| INV| 1-30 Days Past due|Aging___7_Buckets...| 5506.63|
 112535262| FREIGHT| FREIGHT CHARGE| 10.78|02002010001455879...| 020| 1000145587| 95784312| INV| 1-30 Days Past due|Aging___7_Buckets...| 5506.63|
 112535262| SFP-10G-SR=|10GBASE-SR SFP Mo...| 3354.88|02002010001455879...| 020| 1000145587| 95784312| INV| 1-30 Days Past due|Aging___7_Buckets...| 5506.63|
 2462985| A-FLEX-AU-BCCB|AU Meetings Bridg...| 504.0|02002010001476366...| 020| 1000147636| 6100938530| INV| Current|Aging___7_Buckets...| 10143.0|
 2462985| A-FLEX-AUPM1|AU On-Premises Me...| 8568.0|02002010001476366...| 020| 1000147636| 6100938530| INV| Current|Aging___7_Buckets...| 10143.0|
 2462985| A-FLEX-AUCM1|AU Cloud Meetings...| 1071.0|02002010001476366...| 020| 1000147636| 6100938530| INV| Current|Aging___7_Buckets...| 10143.0|
 112380681| GLC-TE=|1000BASE-T SFP tr...| 6322.0|02002010001525619...| 020| 1000152561| 95973562| INV| Current|Aging___7_Buckets...| 54201.52|
 112380681| FREIGHT| FREIGHT CHARGE| 13.1|02002010001525619...| 020| 1000152561| 95973562| INV| Current|Aging___7_Buckets...| 54201.52|
 112380681| GLC-TE=|1000BASE-T SFP tr...| 1580.5|02002010001525619...| 020| 1000152561| 95973562| INV| Current|Aging___7_Buckets...| 54201.52|
 112380681| FREIGHT| FREIGHT CHARGE| 10.91|02002010001525619...| 020| 1000152561| 95973562| INV| Current|Aging___7_Buckets...| 54201.52|
+--------------------+-----------------+--------------------+---------------+--------------------+---------------+---------------+--------------+----------------+--------------------+--------------------+------------------+
only showing top 20 rows

In [0]:
# Join matched PID level data with Flat file by Sales Order
# Create SQL temp table
PID_Inner_df.createOrReplaceTempView("PIDMatch")
# Get service percentage from flag file for each matched PID
PID_Flat = spark.sql("SELECT * \
                      FROM PIDMatch \
                      LEFT JOIN Flat \
                      ON PIDMatch.Sales_Order == Flat.ERPSalesOrderNumber")

# Create SQL temp table
PID_Flat.createOrReplaceTempView("PIDFlat")
PID_Flat.show()

+-----------+-----------------+--------------------+---------------+--------------------+---------------+---------------+--------------+----------------+--------------------+--------------------+------------------+-------------------+-------+-------------+------------+------------+-----------+--------------+------------+-------------+-----------+--------------+-------------+--------------+-------------+---------------+--------------+------------------+-----------------+------------------+-----------------+
Sales_Order| PID| PIDDescription|UnitPriceforPID| Key1|Company_Segment|Customer_number|Invoice_Number|Transaction_Type| Bucket| FileName|Outstanding_Amount|ERPSalesOrderNumber|Deal_ID|20_Carmel_TSS|20_Carmel_AS|20_Cisco_TSS|20_Cisco_AS|21_Carmel_TSS |21_Carmel_AS|21_Cisco_TSS |21_Cisco_AS|20_TSS_Percent|20_AS_Percent|21_TSS_Percent|21_AS_Percent|21Q_TSS_Percent|21Q_AS_Percent|20_TSS_Percent_Tag|20_AS_Percent_Tag|21_TSS_Percent_Tag|21_AS_Percent_Tag|
+-----------+-----------------+--------------------+---------------+--------------------+---------------+---------------+--------------+----------------+--------------------+--------------------+------------------+-------------------+-------+-------------+------------+------------+-----------+--------------+------------+-------------+-----------+--------------+-------------+--------------+-------------+---------------+--------------+------------------+-----------------+------------------+-----------------+
 168481| A-SPK-NU-M3|Business Messagin...| -583.92|55555511170400967...| 555| 111704009| 6700048994| CM| 31-60 Days Past due|Aging___7_Buckets...| -583.92| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null|
 168529| A-SPK-NU-M3|Business Messagin...| 18.59|62462410899965000...| 624| 108999| 6500022911| INV| 361+ Days Past due|Aging___7_Buckets...| 21.57| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null|
 168529|A-SPK-CS-TOLLUSER| Toll Named Users| 2.98|62462410899965000...| 624| 108999| 6500022911| INV| 361+ Days Past due|Aging___7_Buckets...| 21.57| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null|
 205270| CON-SNT-CP8841K9|SNTC-8X5XNBD Cisc...| -13.09|02002010060663971...| 020| 1006066397| 1020783| CM| 361+ Days Past due|Aging___7_Buckets...| -127.68| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null|
 205270| CON-SNT-CP8841K9|SNTC-8X5XNBD Cisc...| -104.72|02002010060663971...| 020| 1006066397| 1020783| CM| 361+ Days Past due|Aging___7_Buckets...| -127.68| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null|
 209423|CON-SSSNT-C11118P|SOLN SUPP 8X5XNBD...| -108.46|020020225131026211CM| 020| 22513| 1026211| CM|91-180 Days Past due|Aging___7_Buckets...| -108.46| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null|
 209955|CON-ECDN-CSQDUADC|ESS WITH 8X5XNBD ...| -938.07|02002010044750141...| 020| 1004475014| 1026819| CM| 1-30 Days Past due|Aging___7_Buckets...| -938.07| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null|
 209992|CON-PSUT-WSC296PS|PRTNR SUP 8X5XNBD...| -272.46|10510514548351007...| 105| 1454835| 1007114| CM| 1-30 Days Past due|Aging___7_Buckets...| -307.88| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null|
 221745| A-WX-NU-TNU+I|Toll Users Named ...| -278.4|55555511336125567...| 555| 113361255| 6700048913| CM| 31-60 Days Past due|Aging___7_Buckets...| -767.9| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null| null|
 221745| A-WX-NU-MTGS200|Webex Meetings 20...| -4

In [0]:
## Check
# Count total SO in joined file
PID_Flat.select('Sales_Order').drop_duplicates().agg({'Sales_Order': 'count'}).show()

# Count matched SO from flat file
PID_Flat.select('Sales_Order', 'ERPSalesOrderNumber').drop_duplicates().where(col('ERPSalesOrderNumber').isNotNull()).agg({'Sales_Order': 'count'}).show()

+------------------+
count(Sales_Order)|
+------------------+
 343728|
+------------------+

+------------------+
count(Sales_Order)|
+------------------+
 12018|
+------------------+

In [0]:
#### Tag PID to different categories, i.e. in scope/out of scope, product/service/adopter, etc.

## 1) Tag carve out entity, in scope vs out of scope ##
Master_PID = spark.read.format("csv") \
        .option("header", "true") \
        .option("delimiter", ",") \
        .option("inferSchema","true") \
        .option("quote", "\"") \
        .option("escape", "\"") \
        .option("encoding", "UTF-8")\
        .load('dbfs:/mnt/sources/616df0dc323eaf657ddff212/616df0de323eaf652bdff213/Master PID List.csv')

# Rename column
Master_PID = Master_PID.select([F.col(col).alias(col.replace('-', '')) for col in Master_PID.columns])
Master_PID = Master_PID.select([F.col(col).alias(col.replace(' ', '_')) for col in Master_PID.columns])

# Create SQL temp table
Master_PID.createOrReplaceTempView("Master")
Master_PID.show()

+-----+----------------+-----+-------+--------------------+--------------+------------------+---------------------+
 PHTG|PHTG_Description| PHBU| PHPF| PHPF_Description|PHProduct_Type| PHPID|PHProduct_Description|
+-----+----------------+-----+-------+--------------------+--------------+------------------+---------------------+
CSPTG| CSPTG|CSPBU| UCS|UCS Servers Unif...| BASE| MSWS-08-EN-P2| ^VOID|
CSPTG| CSPTG|CSPBU|UCS3PSW|3rd party SW for ...| APPSWIND| UCS-BD-MCH-B-1Y| Base LIC MapR Con...|
CSPTG| CSPTG|CSPBU| UCS|UCS Servers Unif...| ACCESSORY| PACK-QSFP-SFP| Packaging for QSF...|
CSPTG| CSPTG|CSPBU| UCSHX|UCSHX HyperFlex H...| SERVER| HXAF220C-M6S=| Cisco HyperFlex H...|
CSPTG| CSPTG|CSPBU| UCSC|UCSC UCS C Series...| ASSEMBLY| UCSC-CP-C240=| ^UCS C240 Control...|
CSPTG| CSPTG|CSPBU| UCSP|UCSP Servers Smar...| SERVER| UCS-EZ8-B200M3-P2| ^UCS SP8 B200M3 P...|
CSPTG| CSPTG|CSPBU| UCSBDI| UCSB WW Disti| ACCESSORY| MDE-1125-K9-M4| MDE-1125-K9= Sola...|
CSPTG| CSPTG|CSPBU| UCSP|UCSP Servers Smar...| SERVER| UCS-SP5-ENSC-B200| ^UCS B200 M3 Blad...|
CSPTG| CSPTG|CSPBU|CLDAUTO| Cloud Automation| APPSWIND| L-TCA3HVP-NP-K9| ^Per Virtualized ...|
CSPTG| CSPTG|CSPBU| UCSP|UCSP Servers Smar...| SERVER| UCS-SPR-C240-P1| ^UCS C240 M3 SFF ...|
CSPTG| CSPTG|CSPBU| UCSMEM|Memory Components...| MEMORY| UCS-MR-X32G2RT-H| 32GB DDR4-2933-MH...|
CSPTG| CSPTG|CSPBU| UCSB|UCSB UCS B Series...| CABLE| UCSBX-C-DEBUGCBL=| UCS BX Compute No...|
CSPTG| CSPTG|CSPBU| UCSC|UCSC UCS C Series...| BASE|UCSC-SW-C480M5-P01| Performance Optim...|
CSPTG| CSPTG|CSPBU| UCSB|UCSB UCS B Series...| BOARD| UCS-CPU-E5-4610E| 1.80 GHz E5-4610 ...|
CSPTG| CSPTG|CSPBU| UCSP|UCSP Servers Smar...| MULTIPROD|UCSB-FPOD-B200S-UD| UCS FLEXPOD DC S ...|
CSPTG| CSPTG|CSPBU| UCSP|UCSP Servers Smar...| SERVER| UCS-SP-C220M5-B2| SP C220 M5SX w/2x...|
CSPTG| CSPTG|CSPBU| UCSP|UCSP Servers Smar...| DEVICES| UCS-EZ7-LSI-400M| ^UCS SP7 LSI 400G...|
CSPTG| CSPTG|CSPBU| UCSHX|UCSHX HyperFlex H...| SERVER|HX-SP-220M4SBP1-3A| ^UCS SP HX220c Hy...|
CSPTG| CSPTG|CSPBU| UCS|UCS Servers Unif...| BOARD| UCS-CPU-E54640B| ^2.2 GHz E5-4640v...|
CSPTG| CSPTG|CSPBU| UCS|UCS Servers Unif...| BASE| RHEL-4S-1G-1A-RS| ^RHEL/4 Socket/1 ...|
+-----+----------------+-----+-------+--------------------+--------------+------------------+---------------------+
only showing top 20 rows

In [0]:
# Read inscope PID file from Blob
Inscope_PID = spark.read.format("csv") \
        .option("header", "true") \
        .option("delimiter", ",") \
        .option("inferSchema","true") \
        .option("quote", "\"") \
        .option("escape", "\"") \
        .option("encoding", "UTF-8")\
        .load('dbfs:/mnt/sources/616f5c03323eaf054fdff222/616f5c05323eafae43dff223/Inscope PID List.csv')

# Create SQL temp table
Inscope_PID.createOrReplaceTempView("Inscope")
Inscope_PID.show()

+------+--------------------+---------------------+-----+-----+------+
 PF | PIDs|Carmel_Intersight_Tag| TG| BU| PF|
+------+--------------------+---------------------+-----+-----+------+
 C1BD| C1BD_ADJ_PROD| In-scope- Carmel|CSPTG|CSPBU| C1BD|
 C1BD| C1A2-1Y-ECSBDA| In-scope- Carmel|CSPTG|CSPBU| C1BD|
 C1BD| C1BD_OTHER_ADJ| In-scope- Carmel|CSPTG|CSPBU| C1BD|
 C1BD|C1BD_Servers_ADJ_...| In-scope- Carmel|CSPTG|CSPBU| C1BD|
 C1BD| C1BD_SW_ADJ_PROD| In-scope- Carmel|CSPTG|CSPBU| C1BD|
 C1BD| C1A2-3Y-ECSBDA| In-scope- Carmel|CSPTG|CSPBU| C1BD|
 C1BD| C1A2-5Y-ECSBDA| In-scope- Carmel|CSPTG|CSPBU| C1BD|
 C1BD| C1BD_CREDIT_ADJ| In-scope- Carmel|CSPTG|CSPBU| C1BD|
C1IAAS| C1-ECS-C02S1| In-scope- Carmel|CSPTG|CSPBU|C1IAAS|
C1IAAS| C1 ECS OPTN| In-scope- Carmel|CSPTG|CSPBU|C1IAAS|
C1IAAS| C1-ECS-C01S2| In-scope- Carmel|CSPTG|CSPBU|C1IAAS|
C1IAAS| C1-ECS-C01-S2| In-scope- Carmel|CSPTG|CSPBU|C1IAAS|
C1IAAS| C1F2-3Y-ECS50| In-scope- Carmel|CSPTG|CSPBU|C1IAAS|
C1IAAS| CWOM-NFR=| In-scope- Carmel|CSPTG|CSPBU|C1IAAS|
C1IAAS| CWOM-751+SVR=| In-scope- Carmel|CSPTG|CSPBU|C1IAAS|
C1IAAS| C1-3Y-SVC-TRK| In-scope- Carmel|CSPTG|CSPBU|C1IAAS|
C1IAAS| ELA2-C1-IA-3Y| In-scope- Carmel|CSPTG|CSPBU|C1IAAS|
C1IAAS| C1-ECS-WOM-751SVR=| In-scope- Carmel|CSPTG|CSPBU|C1IAAS|
C1IAAS| C1-ECS-C02-S1| In-scope- Carmel|CSPTG|CSPBU|C1IAAS|
C1IAAS| C1-ECS-1Y-C02S4| In-scope- Carmel|CSPTG|CSPBU|C1IAAS|
+------+--------------------+---------------------+-----+-----+------+
only showing top 20 rows

In [0]:
# # Tag PID with Carmel/Intersight or Out of scope
# IS_Tag = spark.sql("SELECT * \
#                           ,CASE WHEN m.PHPID IS NOT NULL AND i.PIDs IS NOT NULL THEN Carmel_Intersight_Tag \
#                                 WHEN m.PHPID IS NULL AND p.PID NOT LIKE '%FREIGHT%' THEN 'Out of Scope' \
#                                 ELSE NULL END AS Scope_Tag \
#                     FROM PIDFlat AS p \
#                     LEFT JOIN \
#                         (SELECT DISTINCT PHPID \
#                          FROM Master) AS m \
#                     ON TRIM(p.PID) == TRIM(m.PHPID) \
#                     LEFT JOIN \
#                         (SELECT DISTINCT PIDs, Carmel_Intersight_Tag \
#                          FROM Inscope) AS i \
#                     ON TRIM(p.PID) == TRIM(i.PIDs)")


# IS_Tag.createOrReplaceTempView("ISTag")
# # IS_Tag.show()
# IS_Tag.groupBy('Scope_Tag').agg({'UnitPriceforPID': 'sum'}).show()

In [0]:
# Tag PID with Carmel/Intersight or Out of scope
IS_Tag = spark.sql("SELECT * \
                          ,CASE WHEN i.PIDs IS NOT NULL THEN Carmel_Intersight_Tag \
                                WHEN i.PIDs IS NULL     THEN 'Out of Scope' \
                                ELSE NULL END AS Scope_Tag \
                    FROM PIDFlat AS p \
                    LEFT JOIN \
                        (SELECT DISTINCT PIDs, Carmel_Intersight_Tag \
                         FROM Inscope) AS i \
                    ON TRIM(p.PID) == TRIM(i.PIDs)\
                    WHERE p.PID NOT LIKE '%FREIGHT%' ")

# Create SQL temp table  
IS_Tag.createOrReplaceTempView("ISTag")
# IS_Tag.show()

# Check in scope/out of scope population
IS_Tag.withColumn('Unit Price for PID', col('UnitPriceforPID').cast('int')).groupBy('Scope_Tag').agg({'Unit Price for PID': 'sum'}).show()
IS_Tag.groupBy('Scope_Tag').agg(countDistinct('PID')).show()

+-------------------+-----------------------+
 Scope_Tag|sum(Unit Price for PID)|
+-------------------+-----------------------+
In-scope-Intersight| 294464384|
 Out of Scope| 110377743875|
 In-scope- Carmel| 2785095219|
+-------------------+-----------------------+

+-------------------+----------+
 Scope_Tag|count(PID)|
+-------------------+----------+
In-scope-Intersight| 18|
 Out of Scope| 34824|
 In-scope- Carmel| 1767|
+-------------------+----------+

In [0]:
## 2) Tag service PID ##
Service_PID = spark.read.format("csv") \
        .option("header", "true") \
        .option("delimiter", ",") \
        .option("inferSchema","true") \
        .option("quote", "\"") \
        .option("escape", "\"") \
        .option("encoding", "UTF-8")\
        .load( 'dbfs:/mnt/sources/616f2ffe323eaf69dddff21a/616f3000323eafd1ccdff21b/Service PID.csv')

# Rename column
Service_PID = Service_PID.select([F.col(col).alias(col.replace('-', '')) for col in Service_PID.columns])
Service_PID = Service_PID.select([F.col(col).alias(col.replace(' ', '_')) for col in Service_PID.columns])

# Create SQL temp table
Service_PID.createOrReplaceTempView("Service")
Service_PID.show()

+----------------+--------------------+--------------+------------------+
Technology_Group| Business_Unit|Product_Family| ProductID|
+----------------+--------------------+--------------+------------------+
 SERVICE| ADVANCED SERVICES| NC(AS-TS)30| CON-NCDOS-APICL3|
 SERVICE|TECHNICAL SUPPORT...| LEGACY SNTC| CON-NC2P-SRC220E4|
 SERVICE|TECHNICAL SUPPORT...| COSE-PVSO| CON-PESW-IPEPREAG|
 SERVICE|TECHNICAL SUPPORT...| 3YR SMARTNET| CON-3SNT-B420M3D|
 SERVICE|TECHNICAL SUPPORT...| SNTC| CON-SNTP-C6508DC|
 SERVICE|TECHNICAL SUPPORT...| SNTC| CON-SNT-15454ESFL|
 SERVICE|TECHNICAL SUPPORT...| PM2|CON-P2OP-C3550-24S|
 SERVICE|TECHNICAL SUPPORT...| SSPT PLUS|CON-SSSNT-1783BMSL|
 SERVICE| ADVANCED SERVICES| NC (AS-TS)| CON-NCFE-SNS3415|
 SERVICE|TECHNICAL SUPPORT...| ETSP| CON-ETSSU-SM92X|
 SERVICE|TECHNICAL SUPPORT...| COLL-TS INT| CON-CBNP-5X1GEV2|
 SERVICE|TECHNICAL SUPPORT...| COSE-CBR| CON-ESW-CM6.0SLI|
 SERVICE| ADVANCED SERVICES| NC (AS-TS)| CON-NCF2-N9KC933Q|
 SERVICE|TECHNICAL SUPPORT...| CSSP| CON-CSSPD-210GK9|
 SERVICE|TECHNICAL SUPPORT...| SWAPPS| CON-SAS-CICPT1250|
 SERVICE|TECHNICAL SUPPORT...| SWAPPS| CON-SAS-CSALRG5|
 SERVICE|TECHNICAL SUPPORT...| SNTC| CON-SW9-4500-M|
 SERVICE|TECHNICAL SUPPORT...| 5YR SMARTNET| CON-5SNT-C9120AXE|
 SERVICE|TECHNICAL SUPPORT...| 3YR SMARTNET| CON-3OSP-N7KC7018|
 SERVICE|TECHNICAL SUPPORT...| SWAPPS| CON-SAS-L1VPNCGL|
+----------------+--------------------+--------------+------------------+
only showing top 20 rows

In [0]:
# Tag PID as Service 
Ser_Tag = spark.sql("SELECT * \
                           ,CASE WHEN s.ProductID IS NOT NULL AND Business_Unit =='TECHNICAL SUPPORT SERVICES' THEN 'TS' \
                                 WHEN s.ProductID IS NOT NULL AND Business_Unit =='ADVANCED SERVICES' THEN 'AS' \
                                 ELSE Scope_Tag END AS Scope_Tag2 \
                    FROM ISTag AS i \
                    LEFT JOIN \
                        (SELECT DISTINCT ProductID, Business_Unit\
                         FROM Service \
                         WHERE Business_Unit IN ('TECHNICAL SUPPORT SERVICES', 'ADVANCED SERVICES')) AS s \
                    ON TRIM(i.PID) == TRIM(s.ProductID)")

# Create SQL temp table
Ser_Tag.createOrReplaceTempView("SerTag")

# Check PID tagging breakdown
Ser_Tag.groupBy('Scope_Tag2').agg({'UnitPriceforPID': 'sum'}).show()
Ser_Tag.groupBy('Scope_Tag2').agg(countDistinct('PID')).show()

+-------------------+--------------------+
 Scope_Tag2|sum(UnitPriceforPID)|
+-------------------+--------------------+
In-scope-Intersight| 2.9446478866E8|
 Out of Scope|7.450060277755978E10|
 TS|2.771426555316005...|
 AS| 8.163337118379998E9|
 In-scope- Carmel| 2.78511143702E9|
+-------------------+--------------------+

+-------------------+----------+
 Scope_Tag2|count(PID)|
+-------------------+----------+
In-scope-Intersight| 18|
 Out of Scope| 17372|
 TS| 16581|
 AS| 871|
 In-scope- Carmel| 1767|
+-------------------+----------+

In [0]:
## 3) Tag Adopter PID ##
Adopter_PID = spark.read.format("csv") \
        .option("header", "true") \
        .option("delimiter", ",") \
        .option("inferSchema","true") \
        .option("quote", "\"") \
        .option("escape", "\"") \
        .option("encoding", "UTF-8")\
        .load('dbfs:/mnt/sources/616f374d323eafbc30dff21e/616f3762323eaff7f3dff21f/Adopter PID.csv')

# Rename column
Adopter_PID = Adopter_PID.select([F.col(col).alias(col.replace('-', '')) for col in Adopter_PID.columns])
Adopter_PID = Adopter_PID.select([F.col(col).alias(col.replace(' ', '_')) for col in Adopter_PID.columns])

# Create SQL temp table
Adopter_PID.createOrReplaceTempView("Adopter")
Adopter_PID.show()

+---------------------------+------------------+------------------+-----------+
Sanjuvan's_Revenue_PID_List| Spend_PID_List| PID|AccountType|
+---------------------------+------------------+------------------+-----------+
 UCSC-C240-CTR|LIC-CT8540-DTLS-K9| UCSC-C240-CTR| Revenue|
 ST-DNODE-G1| AIR-CT8540-K9| ST-DNODE-G1| Revenue|
 ST-DS6200-1D-CH-K9| AIR-CT6880-NIC-K9|ST-DS6200-1D-CH-K9| Revenue|
 ST-DS6200-1D-K9| AIR-MR-1X161RV-A| ST-DS6200-1D-K9| Revenue|
 ST-DS6200-1D-K9=| AIR-MRAID12G| ST-DS6200-1D-K9=| Revenue|
 ST-DS6200-K9| AIR-MRAID12G-1GB| ST-DS6200-K9| Revenue|
 LC-FC-5020D-NF| AIR-BZL-C240M4| LC-FC-5020D-NF| Revenue|
 LC-FC-5020D-NF=| AIR-CPU-E52680D| LC-FC-5020D-NF=| Revenue|
 LC-FC-5020E-NF| AIR-PCI-1A-240M4| LC-FC-5020E-NF| Revenue|
 LC-FC-5020E-NF=| AIR-PSU2V2-1200W| LC-FC-5020E-NF=| Revenue|
 LC-FC-NF-5020-K9| AIR-SD240GBKS4-EV| LC-FC-NF-5020-K9| Revenue|
 ST-FC4200-CHAS-K9| AIR-SD-32G-S| ST-FC4200-CHAS-K9| Revenue|
 ST-FC4200-K9| AIR-TPM2-001| ST-FC4200-K9| Revenue|
 ST-FC4210-CHAS-K9| NXA-PAC-500W-PE| ST-FC4210-CHAS-K9| Revenue|
 ST-FC4210-K9| FPR9K-SM-48=| ST-FC4210-K9| Revenue|
 ST-FC5200-D| SNS-3615-K9| ST-FC5200-D| Revenue|
 ST-FC5200D-CHAS-K9| SNS-PSU1-770W|ST-FC5200D-CHAS-K9| Revenue|
 ST-FC5200-E| SNS-CPU-4110| ST-FC5200-E| Revenue|
 ST-FC5200E-CHAS-K9| SNS-HD600G10K12N|ST-FC5200E-CHAS-K9| Revenue|
 ST-FC5200-K9| SNS-MR-X16G1RT-H| ST-FC5200-K9| Revenue|
+---------------------------+------------------+------------------+-----------+
only showing top 20 rows

In [0]:
# Tag PID as Adopter
Adopt_Tag = spark.sql("SELECT * \
                            ,CASE WHEN a.PID IS NOT NULL THEN 'Adopter' ELSE Scope_Tag2 END AS Scope_Tag3 \
                    FROM SerTag AS s \
                    LEFT JOIN \
                        (SELECT DISTINCT PID\
                         FROM Adopter) AS a \
                    ON TRIM(s.PID) == TRIM(a.PID)")

# Create SQL temp table
Adopt_Tag.createOrReplaceTempView("AdpTag")

# Check PID tagging breakdown
Adopt_Tag.groupBy('Scope_Tag3').agg({'UnitPriceforPID': 'sum'}).show()
Adopt_Tag.groupBy('Scope_Tag3').agg(countDistinct('s.PID')).show()

+-------------------+--------------------+
 Scope_Tag3|sum(UnitPriceforPID)|
+-------------------+--------------------+
In-scope-Intersight| 2.9446478866E8|
 Adopter| 2.041519078260001E9|
 Out of Scope|7.245908369929985E10|
 TS|2.771426555316005...|
 AS| 8.163337118379993E9|
 In-scope- Carmel| 2.785111437019999E9|
+-------------------+--------------------+

+-------------------+------------+
 Scope_Tag3|count(s.PID)|
+-------------------+------------+
In-scope-Intersight| 18|
 Adopter| 207|
 Out of Scope| 17165|
 TS| 16581|
 AS| 871|
 In-scope- Carmel| 1767|
+-------------------+------------+

In [0]:
## Product Percentage is to divide unit price for each product ID by total unit price for each key1 level item (invoice number in this case) ##
# Calculate product unit price per PID and per invoice
Prod_UP_df = spark.sql("SELECT a.Key1, Company_Segment, Customer_number, Invoice_Number, Transaction_Type, Scope_Tag3 \
                               ,SUM(UnitPriceforPID) AS UP, Total_UP \
                        FROM AdpTag AS a \
                        INNER JOIN \
                             (SELECT Key1, SUM(UnitPriceforPID) AS Total_UP \
                              FROM AdpTag \
                              GROUP BY Key1) AS b \
                        ON a.Key1 == b.Key1 \
                        GROUP BY a.Key1, Company_Segment, Customer_number, Invoice_Number, Transaction_Type, Scope_Tag3, Total_UP \
                        HAVING Scope_Tag3 NOT IN ('TS','AS') ")

# Create SQL temp table
Prod_UP_df.createOrReplaceTempView("PUP")
Prod_UP_df.show()

+--------------------+---------------+---------------+--------------+----------------+------------+------------------+------------------+
 Key1|Company_Segment|Customer_number|Invoice_Number|Transaction_Type| Scope_Tag3| UP| Total_UP|
+--------------------+---------------+---------------+--------------+----------------+------------+------------------+------------------+
02002010000414696...| 020| 1000041469| 6100943780| INV|Out of Scope| 225.6| 225.6|
02002010000813149...| 020| 1000081314| 95416437| INV|Out of Scope| 3236.8| 3236.8|
02002010000954678...| 020| 1000095467| 80393823| CM|Out of Scope| -26.09| -26.09|
02002010000954679...| 020| 1000095467| 95981137| INV|Out of Scope| 349.22| 349.22|
02002010000968819...| 020| 1000096881| 95966259| INV|Out of Scope| 3886.0| 3886.0|
02002010001038796...| 020| 1000103879| 6100770671| INV|Out of Scope| 1162.2| 1162.2|
02002010001455879...| 020| 1000145587| 95784312| INV|Out of Scope| 5125.04| 5125.04|
02002010001476366...| 020| 1000147636| 6100938530| INV|Out of Scope| 10143.0| 10143.0|
02002010001525619...| 020| 1000152561| 95973562| INV|Out of Scope| 54123.86| 54123.86|
02002010001526126...| 020| 1000152612| 6100933526| INV|Out of Scope|435.96000000000004|435.96000000000004|
02002010001636306...| 020| 1000163630| 6100933553| INV|Out of Scope| 790.5999999999999| 790.5999999999999|
02002010001636306...| 020| 1000163630| 6100933558| INV|Out of Scope| 3114.0| 3114.0|
02002010001636306...| 020| 1000163630| 6100941833| INV|Out of Scope| 1356.09| 1356.09|
02002010001751046...| 020| 1000175104| 6100290488| INV|Out of Scope| 19500.0| 19500.0|
02002010001751046...| 020| 1000175104| 6100933614| INV|Out of Scope| 2123.4| 2123.4|
02002010001814469...| 020| 1000181446| 95891188| INV|Out of Scope| 400.0| 400.0|
02002010001814469...| 020| 1000181446| 95961393| INV|Out of Scope| 2000.0| 2000.0|
02002010001814469...| 020| 1000181446| 96020450| INV|Out of Scope| 400.0| 400.0|
02002010001829619...| 020| 1000182961| 95884220| INV|Out of Scope| 18408.0| 18408.0|
02002010002312286...| 020| 1000231228| 6100787009| INV|Out of Scope| 126.0| 126.0|
+--------------------+---------------+---------------+--------------+----------------+------------+------------------+------------------+
only showing top 20 rows

In [0]:
# Calculate product in scope % at invoice level
Prod_Percent = spark.sql("SELECT * \
                                ,CASE WHEN Scope_Tag3 == 'In-scope- Carmel'    THEN UP/NULLIF(Total_UP,0) ELSE 0 END AS Inscope_Carmel_Percent_Invoice \
                                ,CASE WHEN Scope_Tag3 == 'In-scope-Intersight' THEN UP/NULLIF(Total_UP,0) ELSE 0 END AS Inscope_Intersight_Percent_Invoice \
                                ,CASE WHEN Scope_Tag3 == 'Adopter'             THEN UP/NULLIF(Total_UP,0) ELSE 0 END AS Inscope_Adopter_Percent_Invoice \
                          FROM PUP")

# Create SQL temp table
Prod_Percent.createOrReplaceTempView("PPT")
Prod_Percent.show()

+--------------------+---------------+---------------+--------------+----------------+------------+------------------+------------------+------------------------------+----------------------------------+-------------------------------+
 Key1|Company_Segment|Customer_number|Invoice_Number|Transaction_Type| Scope_Tag3| UP| Total_UP|Inscope_Carmel_Percent_Invoice|Inscope_Intersight_Percent_Invoice|Inscope_Adopter_Percent_Invoice|
+--------------------+---------------+---------------+--------------+----------------+------------+------------------+------------------+------------------------------+----------------------------------+-------------------------------+
02002010000414696...| 020| 1000041469| 6100943780| INV|Out of Scope| 225.6| 225.6| 0.0| 0.0| 0.0|
02002010000813149...| 020| 1000081314| 95416437| INV|Out of Scope| 3236.8| 3236.8| 0.0| 0.0| 0.0|
02002010000954678...| 020| 1000095467| 80393823| CM|Out of Scope| -26.09| -26.09| 0.0| 0.0| 0.0|
02002010000954679...| 020| 1000095467| 95981137| INV|Out of Scope| 349.22| 349.22| 0.0| 0.0| 0.0|
02002010000968819...| 020| 1000096881| 95966259| INV|Out of Scope| 3886.0| 3886.0| 0.0| 0.0| 0.0|
02002010001038796...| 020| 1000103879| 6100770671| INV|Out of Scope| 1162.2| 1162.2| 0.0| 0.0| 0.0|
02002010001455879...| 020| 1000145587| 95784312| INV|Out of Scope| 5125.04| 5125.04| 0.0| 0.0| 0.0|
02002010001476366...| 020| 1000147636| 6100938530| INV|Out of Scope| 10143.0| 10143.0| 0.0| 0.0| 0.0|
02002010001525619...| 020| 1000152561| 95973562| INV|Out of Scope| 54123.86| 54123.86| 0.0| 0.0| 0.0|
02002010001526126...| 020| 1000152612| 6100933526| INV|Out of Scope|435.96000000000004|435.96000000000004| 0.0| 0.0| 0.0|
02002010001636306...| 020| 1000163630| 6100933553| INV|Out of Scope| 790.5999999999999| 790.5999999999999| 0.0| 0.0| 0.0|
02002010001636306...| 020| 1000163630| 6100933558| INV|Out of Scope| 3114.0| 3114.0| 0.0| 0.0| 0.0|
02002010001636306...| 020| 1000163630| 6100941833| INV|Out of Scope| 1356.09| 1356.09| 0.0| 0.0| 0.0|
02002010001751046...| 020| 1000175104| 6100290488| INV|Out of Scope| 19500.0| 19500.0| 0.0| 0.0| 0.0|
02002010001751046...| 020| 1000175104| 6100933614| INV|Out of Scope| 2123.4| 2123.4| 0.0| 0.0| 0.0|
02002010001814469...| 020| 1000181446| 95891188| INV|Out of Scope| 400.0| 400.0| 0.0| 0.0| 0.0|
02002010001814469...| 020| 1000181446| 95961393| INV|Out of Scope| 2000.0| 2000.0| 0.0| 0.0| 0.0|
02002010001814469...| 020| 1000181446| 96020450| INV|Out of Scope| 400.0| 400.0| 0.0| 0.0| 0.0|
02002010001829619...| 020| 1000182961| 95884220| INV|Out of Scope| 18408.0| 18408.0| 0.0| 0.0| 0.0|
02002010002312286...| 020| 1000231228| 6100787009| INV|Out of Scope| 126.0| 126.0| 0.0| 0.0| 0.0|
+--------------------+---------------+---------------+--------------+----------------+------------+------------------+------------------+------------------------------+----------------------------------+-------------------------------+
only showing top 20 rows

In [0]:
## Service Percentage is to divide unit price for each service ID by total unit price for each key1 level item (invoice number in this case) ##
# Calculate service unit price per PID and per invoice
Serv_UP_df = spark.sql("SELECT a.Key1, Company_Segment, Customer_number, Invoice_Number, Transaction_Type, Scope_Tag3 \
                               ,CASE WHEN Scope_Tag3 == 'TS' THEN SUM(21_TSS_Percent * UnitPriceforPID) ELSE 0 END AS TSS_UP \
                               ,CASE WHEN Scope_Tag3 == 'AS' THEN SUM(21_AS_Percent * UnitPriceforPID)  ELSE 0 END AS AS_UP \
                               ,Total_UP \
                        FROM AdpTag AS a \
                        INNER JOIN \
                             (SELECT Key1, SUM(UnitPriceforPID) AS Total_UP \
                              FROM AdpTag \
                              GROUP BY Key1) AS b \
                        ON a.Key1 == b.Key1 \
                        GROUP BY a.Key1, Company_Segment, Customer_number, Invoice_Number, Transaction_Type, Scope_Tag3, Total_UP \
                        HAVING Scope_Tag3 IN ('TS','AS') ")

# Create SQL temp table
Serv_UP_df.createOrReplaceTempView("SUP")
Serv_UP_df.show()

+--------------------+---------------+---------------+--------------+----------------+----------+------------------+-----+-----------------+
 Key1|Company_Segment|Customer_number|Invoice_Number|Transaction_Type|Scope_Tag3| TSS_UP|AS_UP| Total_UP|
+--------------------+---------------+---------------+--------------+----------------+----------+------------------+-----+-----------------+
02002010001205191...| 020| 1000120519| 1000083098| CM| TS| null| 0.0| -87.91|
02002010001526129...| 020| 1000152612| 95885277| INV| TS| null| 0.0| 11870.72|
02002010002162869...| 020| 1000216286| 95934706| INV| TS| null| 0.0| 1645.19|
02002010002449199...| 020| 1000244919| 95899500| INV| TS| null| 0.0| 2369.92|
02002010002449199...| 020| 1000244919| 95899695| INV| TS| null| 0.0| 8962.21|
02002010002449199...| 020| 1000244919| 95908229| INV| TS| null| 0.0| 4105.65|
02002010002449199...| 020| 1000244919| 95952179| INV| TS| null| 0.0| 1300.4|
02002010002449199...| 020| 1000244919| 95967914| INV| TS| null| 0.0| 2570.8|
02002010002449199...| 020| 1000244919| 95993710| INV| TS| null| 0.0| 304.8|
02002010002449199...| 020| 1000244919| 96005789| INV| TS| null| 0.0| 3539.37|
02002010002567029...| 020| 1000256702| 95877555| INV| TS| null| 0.0| 1382.4|
02002010002567029...| 020| 1000256702| 95903359| INV| TS| null| 0.0| 17861.76|
02002010002613999...| 020| 1000261399| 95984034| INV| AS| 0.0| null| 84883.02|
02002010002785489...| 020| 1000278548| 96017048| INV| TS| null| 0.0| 139.05|
02002010002840076...| 020| 1000284007| 6100928226| INV| TS| null| 0.0| 231399.0|
02002010003014249...| 020| 1000301424| 94953148| INV| TS|54.695800000000006| 0.0|5469.580000000001|
02002010044750149...| 020| 1004475014| 95872217| INV| TS| null| 0.0| 830.95|
02002010044750149...| 020| 1004475014| 95876915| INV| TS| 24.8455| 0.0| 496.91|
02002010044750149...| 020| 1004475014| 95877195| INV| TS| null| 0.0| 423.5|
02002010044750149...| 020| 1004475014| 95911859| INV| TS| null| 0.0| 2238.0|
+--------------------+---------------+---------------+--------------+----------------+----------+------------------+-----+-----------------+
only showing top 20 rows

In [0]:
# Calculate service in scope % at invoice level
Serv_Percent = spark.sql("SELECT * \
                                ,CASE WHEN Scope_Tag3 == 'TS' THEN TSS_UP/NULLIF(Total_UP,0) ELSE 0 END AS TS_Percent_Invoice \
                                ,CASE WHEN Scope_Tag3 == 'AS' THEN AS_UP/NULLIF(Total_UP,0)  ELSE 0 END AS AS_Percent_Invoice \
                          FROM SUP ")

# Create SQL temp table
Serv_Percent.createOrReplaceTempView("SPT")
Serv_Percent.show()

+--------------------+---------------+---------------+--------------+----------------+----------+------------------+-----+-----------------+------------------+------------------+
 Key1|Company_Segment|Customer_number|Invoice_Number|Transaction_Type|Scope_Tag3| TSS_UP|AS_UP| Total_UP|TS_Percent_Invoice|AS_Percent_Invoice|
+--------------------+---------------+---------------+--------------+----------------+----------+------------------+-----+-----------------+------------------+------------------+
02002010001205191...| 020| 1000120519| 1000083098| CM| TS| null| 0.0| -87.91| null| 0.0|
02002010001526129...| 020| 1000152612| 95885277| INV| TS| null| 0.0| 11870.72| null| 0.0|
02002010002162869...| 020| 1000216286| 95934706| INV| TS| null| 0.0| 1645.19| null| 0.0|
02002010002449199...| 020| 1000244919| 95899500| INV| TS| null| 0.0| 2369.92| null| 0.0|
02002010002449199...| 020| 1000244919| 95899695| INV| TS| null| 0.0| 8962.21| null| 0.0|
02002010002449199...| 020| 1000244919| 95908229| INV| TS| null| 0.0| 4105.65| null| 0.0|
02002010002449199...| 020| 1000244919| 95952179| INV| TS| null| 0.0| 1300.4| null| 0.0|
02002010002449199...| 020| 1000244919| 95967914| INV| TS| null| 0.0| 2570.8| null| 0.0|
02002010002449199...| 020| 1000244919| 95993710| INV| TS| null| 0.0| 304.8| null| 0.0|
02002010002449199...| 020| 1000244919| 96005789| INV| TS| null| 0.0| 3539.37| null| 0.0|
02002010002567029...| 020| 1000256702| 95877555| INV| TS| null| 0.0| 1382.4| null| 0.0|
02002010002567029...| 020| 1000256702| 95903359| INV| TS| null| 0.0| 17861.76| null| 0.0|
02002010002613999...| 020| 1000261399| 95984034| INV| AS| 0.0| null| 84883.02| 0.0| null|
02002010002785489...| 020| 1000278548| 96017048| INV| TS| null| 0.0| 139.05| null| 0.0|
02002010002840076...| 020| 1000284007| 6100928226| INV| TS| null| 0.0| 231399.0| null| 0.0|
02002010003014249...| 020| 1000301424| 94953148| INV| TS|54.695800000000006| 0.0|5469.580000000001| 0.01| 0.0|
02002010044750149...| 020| 1004475014| 95872217| INV| TS| null| 0.0| 830.95| null| 0.0|
02002010044750149...| 020| 1004475014| 95876915| INV| TS| 24.8455| 0.0| 496.91| 0.05| 0.0|
02002010044750149...| 020| 1004475014| 95877195| INV| TS| null| 0.0| 423.5| null| 0.0|
02002010044750149...| 020| 1004475014| 95911859| INV| TS| null| 0.0| 2238.0| null| 0.0|
+--------------------+---------------+---------------+--------------+----------------+----------+------------------+-----+-----------------+------------------+------------------+
only showing top 20 rows

In [0]:
#### Combine product percentage with service percentage at key1 level ####

Prod_Serv_Percent = spark.sql("SELECT u.Key1 \
                                     ,u.Company_Segment \
                                     ,u.Customer_number \
                                     ,u.Invoice_Number \
                                     ,u.Transaction_Type \
                                     ,IF(SUM(u.Total_UP) IS NULL,0,SUM(u.Total_UP)) AS UP \
                                     ,IF(SUM(u.Inscope_Carmel_Percent_Invoice) IS NULL,0,SUM(u.Inscope_Carmel_Percent_Invoice))           AS Inscope_Carmel_Percent_Invoice \
                                     ,IF(SUM(u.Inscope_Intersight_Percent_Invoice) IS NULL,0,SUM(u.Inscope_Intersight_Percent_Invoice))   AS Inscope_Intersight_Percent_Invoice\
                                     ,IF(SUM(u.Inscope_Adopter_Percent_Invoice) IS NULL,0,SUM(u.Inscope_Adopter_Percent_Invoice))         AS Inscope_Adopter_Percent_Invoice\
                                     ,IF(SUM(u.TS_Percent_Invoice) IS NULL,0,SUM(u.TS_Percent_Invoice))                                   AS TS_Percent_Invoice\
                                     ,IF(SUM(u.AS_Percent_Invoice) IS NULL,0,SUM(u.AS_Percent_Invoice))                                   AS AS_Percent_Invoice\
                               FROM \
                                   (SELECT Key1 \
                                          ,Company_Segment \
                                          ,Customer_number \
                                          ,Invoice_Number \
                                          ,Transaction_Type \
                                          ,0 AS TSS_UP \
                                          ,0 AS AS_UP \
                                          ,UP AS Total_UP\
                                          ,0 AS TS_Percent_Invoice \
                                          ,0 AS AS_Percent_Invoice \
                                          ,Inscope_Carmel_Percent_Invoice \
                                          ,Inscope_Intersight_Percent_Invoice \
                                          ,Inscope_Adopter_Percent_Invoice \
                                   FROM PPT AS p\
                                   UNION \
                                   SELECT  Key1 \
                                          ,Company_Segment \
                                          ,Customer_number \
                                          ,Invoice_Number \
                                          ,Transaction_Type \
                                          ,TSS_UP \
                                          ,AS_UP \
                                          ,Total_UP \
                                          ,TS_Percent_Invoice \
                                          ,AS_Percent_Invoice \
                                          ,0 AS Inscope_Carmel_Percent_Invoice \
                                          ,0 AS Inscope_Intersight_Percent_Invoice \
                                          ,0 AS Inscope_Adopter_Percent_Invoice \
                                   FROM SPT AS s) AS u \
                             GROUP BY u.Key1, u.Company_Segment, u.Customer_number, u.Invoice_Number, u.Transaction_Type \
                             ")

# Calculate out of scope percentage by excluding all product and service percentage
Prod_Serv_Percent = Prod_Serv_Percent.withColumn('Out_of_scope_Percent', lit(1) - col('TS_Percent_Invoice') - col('AS_Percent_Invoice') - col('Inscope_Carmel_Percent_Invoice') - col('Inscope_Intersight_Percent_Invoice') - col('Inscope_Adopter_Percent_Invoice'))

# Create SQL temp table
Prod_Serv_Percent.createOrReplaceTempView("PSPT")
Prod_Serv_Percent.show()

+--------------------+---------------+---------------+--------------+----------------+---------+------------------------------+----------------------------------+-------------------------------+------------------+------------------+--------------------+
 Key1|Company_Segment|Customer_number|Invoice_Number|Transaction_Type| UP|Inscope_Carmel_Percent_Invoice|Inscope_Intersight_Percent_Invoice|Inscope_Adopter_Percent_Invoice|TS_Percent_Invoice|AS_Percent_Invoice|Out_of_scope_Percent|
+--------------------+---------------+---------------+--------------+----------------+---------+------------------------------+----------------------------------+-------------------------------+------------------+------------------+--------------------+
62462455556410399...| 624| 555564103| 9995694783| INV| 2030.0| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
02002040186309861...| 020| 401863098| 6100942184| INV| 7677.6| 0.0| 1.0| 0.0| 0.0| 0.0| 0.0|
62462411471867099...| 624| 114718670| 9995707356| INV| 649.6| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
62462411470292999...| 624| 114702929| 9995695195| INV| 1305.0| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
69669610127077217...| 696| 1012707721| 700236006| INV| 16065.0| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
62462410069708999...| 624| 100697089| 9995633250| INV| 174.0| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
02002010162654655...| 020| 1016265465| 5100448698| INV| 525.0| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
62462410278511755...| 624| 1027851175| 5500092876| INV| 706.6| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
19819811370618611...| 198| 113706186| 11100025285| INV|3726162.0| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
02002010357736100...| 020| 1035773| 6100932412| INV| 876.0| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
90590540031546085...| 905| 400315460| 8509171| INV|111751.19| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
02002075038361009...| 020| 750383| 6100922087| INV| 177.0| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
02002040043410495...| 020| 400434104| 95930326| INV| 2754.47| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
02002010002449196...| 020| 1000244919| 6100937067| INV| 75000.0| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
11511533822964005...| 115| 33822| 9640050| INV| 25647.22| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
02002052549795972...| 020| 525497| 95972480| INV| 14919.92| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
62462410003069979...| 624| 1000306997| 9995656984| INV| 42.14| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
19819811170401311...| 198| 111704013| 11100062141| INV|362161.05| 1.0| 0.0| 0.0| 0.0| 0.0| 0.0|
90590540031546085...| 905| 400315460| 8516016| INV| 22313.6| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
02002064119599454...| 020| 6411| 95994541| INV| 76.72| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
+--------------------+---------------+---------------+--------------+----------------+---------+------------------------------+----------------------------------+-------------------------------+------------------+------------------+--------------------+
only showing top 20 rows

In [0]:
# Check if out of scope percentage is less than 0
spark.sql("SELECT COUNT(*) FROM PSPT WHERE Out_of_scope_Percent<0")

Out[42]:

count(1)
116


In [0]:
spark.sql("SELECT SUM(UP) FROM PSPT WHERE Out_of_scope_Percent<0")

Out[67]:

sum(UP)
2.6055236380000006E7


In [0]:
## Apply Invoice Percentage to Outstanding Amount from text/invoice data to get level1 AR allocation amount
Level1_Allocation = spark.sql("SELECT a.* \
                                   ,OA*Inscope_Carmel_Percent_Invoice      AS Inscope_Carmel_OA_L1 \
                                   ,OA*Inscope_Intersight_Percent_Invoice  AS Inscope_Intersight_OA_L1 \
                                   ,OA*Inscope_Adopter_Percent_Invoice     AS Inscope_Adopter_OA_L1 \
                                   ,OA*TS_Percent_Invoice                  AS Inscope_TS_OA_L1 \
                                   ,OA*AS_Percent_Invoice                  AS Inscope_AS_OA_L1 \
                                   ,OA*Out_of_scope_Percent                AS Outofscope_OA_L1 \
                                   ,CASE WHEN Inscope_Carmel_Percent_Invoice + Inscope_Intersight_Percent_Invoice ==1 THEN 'All In Scope' \
                                         WHEN Out_of_scope_Percent ==1 THEN 'All Out of Scope' \
                                         ELSE 'Mix of Scope' END AS Scope_Category \
                             FROM( \
                                   SELECT p.*, t.OA \
                                   FROM PSPT AS p \
                                   LEFT JOIN \
                                      (SELECT {}, Company_Segment, Transaction_Type, Bucket, SUM(OA) AS OA \
                                       FROM TXTAgg2 \
                                       GROUP BY {}, Company_Segment, Transaction_Type, Bucket) AS t \
                                  ON p.Key1 == t.{} \
                                  ) AS a \
                            ".format(TXT_key1,TXT_key1,TXT_key1))

Level1_Allocation.createOrReplaceTempView("L1_Allocation")
Level1_Allocation.show()

+--------------------+---------------+---------------+--------------+----------------+------------------+------------------------------+----------------------------------+-------------------------------+------------------+------------------+--------------------+---------+--------------------+------------------------+---------------------+----------------+----------------+----------------+----------------+
 Key1|Company_Segment|Customer_number|Invoice_Number|Transaction_Type| UP|Inscope_Carmel_Percent_Invoice|Inscope_Intersight_Percent_Invoice|Inscope_Adopter_Percent_Invoice|TS_Percent_Invoice|AS_Percent_Invoice|Out_of_scope_Percent| OA|Inscope_Carmel_OA_L1|Inscope_Intersight_OA_L1|Inscope_Adopter_OA_L1|Inscope_TS_OA_L1|Inscope_AS_OA_L1|Outofscope_OA_L1| Scope_Category|
+--------------------+---------------+---------------+--------------+----------------+------------------+------------------------------+----------------------------------+-------------------------------+------------------+------------------+--------------------+---------+--------------------+------------------------+---------------------+----------------+----------------+----------------+----------------+
90590579188852173...| 905| 79188| 8521732| INV| 1459.04| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0| 177899.0| 0.0| 0.0| 0.0| 0.0| 0.0| 177899.0|All Out of Scope|
62462411255634899...| 624| 112556348| 9995181992| INV| 36442.0| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0| 44360.11| 0.0| 0.0| 0.0| 0.0| 0.0| 44360.11|All Out of Scope|
62462411171608299...| 624| 111716082| 9995651828| INV| 38000.0| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0| 39048.8| 0.0| 0.0| 0.0| 0.0| 0.0| 39048.8|All Out of Scope|
02002054334895996...| 020| 543348| 95996113| INV|41995.340000000004| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0| 27440.82| 0.0| 0.0| 0.0| 0.0| 0.0| 27440.82|All Out of Scope|
02002025877803972...| 020| 25877| 80397298| CM| -20.22| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0| -20.22| -0.0| -0.0| -0.0| -0.0| -0.0| -20.22|All Out of Scope|
02002010140959514...| 020| 10140| 95951464| INV| 1164.06| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0| 1164.06| 0.0| 0.0| 0.0| 0.0| 0.0| 1164.06|All Out of Scope|
02002010140959641...| 020| 10140| 95964109| INV| 486.07| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0| 486.07| 0.0| 0.0| 0.0| 0.0| 0.0| 486.07|All Out of Scope|
67167110313610154...| 671| 1031361015| 4440173937| INV| 187.6| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0| 228.87| 0.0| 0.0| 0.0| 0.0| 0.0| 228.87|All Out of Scope|
62462410250464019...| 624| 1025046401| 9995645464| INV| 63407.28| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0| 65157.32| 0.0| 0.0| 0.0| 0.0| 0.0| 65157.32|All Out of Scope|
02002036823958923...| 020| 36823| 95892367| INV| 186.54| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0| 186.54| 0.0| 0.0| 0.0| 0.0| 0.0| 186.54|All Out of Scope|
02002013486960065...| 020| 13486| 96006519| INV| 677.36| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0| 677.36| 0.0| 0.0| 0.0| 0.0| 0.0| 677.36|All Out of Scope|
02002010358098255...| 020| 1035809825| 5100422925| INV| 95.0| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0| 100.7| 0.0| 0.0| 0.0| 0.0| 0.0| 100.7|All Out of Scope|
02002011115519594...| 020| 1111551| 95940252| INV| 890.64| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0| 901.92| 0.0| 0.0| 0.0| 0.0| 0.0| 901.92|All Out of Scope|
62462410015486369...| 624| 1001548636| 9995607694| INV|2550.0299999999997| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0| 1384.23| 0.0| 0.0| 0.0| 0.0| 0.0| 1384.23|All Out of Scope|
19819810266940181...| 198| 1026694018| 11100053161| INV| 1917.74| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0| 174.46| 0.0| 0.0| 0.0| 0.0| 0.0| 174.46|All Out of Scope|
62462411078950499...| 624| 110789504| 9995700290| INV| 15346.56| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0| 15346.56| 0.0| 0.0| 0.0| 15346.56| 0.0| 0.0| Mix of Scope|
02002010002449199...| 020| 1000244919| 95986260| INV| 27845.36| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0| 27845.36| 0.0| 0.0| 0.0| 27845.36| 0.0| 0.0| Mix of Scope|
90590540031546085...| 905| 400315460| 8519326| INV| 42.8| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0| 5218.0| 0.0| 0.0| 0.0| 0.0| 0.0| 5218.0|All Out of Scope|
19819882008764811...| 198| 820087648| 11100070353| INV| 503831.38| 0.0| 0.0| 0.0| 0

In [0]:
# Check scope category breakdown
spark.sql("SELECT Scope_Category, SUM(Inscope_Carmel_OA_L1) FROM L1_Allocation GROUP BY Scope_Category")

Out[51]:

Scope_Category,sum(Inscope_Carmel_OA_L1)
Mix of Scope,4.780939033524184E8
All Out of Scope,0.0
All In Scope,3.149882461006547E9


In [0]:
# Check Level1 allocation result
spark.sql("SELECT ROUND(SUM(Inscope_Carmel_OA_L1),2), ROUND(SUM(Inscope_Intersight_OA_L1),2), ROUND(SUM(Inscope_Adopter_OA_L1),2), ROUND(SUM(Inscope_TS_OA_L1),2), ROUND(SUM(Inscope_AS_OA_L1),2), ROUND(SUM(Outofscope_OA_L1),2) FROM L1_Allocation ")

Out[53]:

"round(sum(Inscope_Carmel_OA_L1), 2)","round(sum(Inscope_Intersight_OA_L1), 2)","round(sum(Inscope_Adopter_OA_L1), 2)","round(sum(Inscope_TS_OA_L1), 2)","round(sum(Inscope_AS_OA_L1), 2)","round(sum(Outofscope_OA_L1), 2)"
3.62797636436E9,1.5882947152E8,2.6445618781E9,5.4833432499E8,1.3531315239E8,1.2297024848092E11


In [0]:
# Check Level1 allocation result at entity level
spark.sql("SELECT Company_Segment, ROUND(SUM(Inscope_Carmel_OA_L1),2), ROUND(SUM(Inscope_Intersight_OA_L1),2), ROUND(SUM(Inscope_Adopter_OA_L1),2), ROUND(SUM(Inscope_TS_OA_L1),2), ROUND(SUM(Inscope_AS_OA_L1),2), ROUND(SUM(Outofscope_OA_L1),2) FROM L1_Allocation GROUP BY Company_Segment ORDER BY Company_Segment ASC")

Out[54]:

Company_Segment,"round(sum(Inscope_Carmel_OA_L1), 2)","round(sum(Inscope_Intersight_OA_L1), 2)","round(sum(Inscope_Adopter_OA_L1), 2)","round(sum(Inscope_TS_OA_L1), 2)","round(sum(Inscope_AS_OA_L1), 2)","round(sum(Outofscope_OA_L1), 2)"
020,2.074588545E8,3.900807297E7,3.012416016E7,3.354222099E7,2722615.99,3.34335998293E9
105,1.468555532E7,242825.14,4192252.89,3028184.32,61810.04,2.6071215602E8
115,1.691798598E7,583804.54,5198641.89,2351410.16,85042.61,2.789185022E8
130,9.6045934272E8,3.7320712E7,1.8216747813E9,1.7731292212E8,0.0,6.129796246786E10
193,2.856160658E7,607870.73,233461.6,2183795.46,0.0,2.7115134697E8
198,1.20090483041E9,6.753637806E7,3.3927684384E8,1.9277421002E8,7.034295711E7,2.050626523466E10
260,1.589638678E7,0.0,8150417.45,1492459.87,0.0,5.4041228078E8
263,4301.01,182791.11,197757.7,5414.37,0.0,7.193852315E7
517,0.0,0.0,0.0,0.0,0.0,1.866334522E7
555,4014193.01,79484.85,1197805.29,519857.7,778119.19,7.75692784E7


In [0]:
# Create input parameter
dbutils.widgets.dropdown("PID Key Level 2",Level1_Allocation.columns[0], Level1_Allocation.columns)

dbutils.widgets.get("PID Key Level 2")

Out[107]: 'Customer_number'

In [0]:
# Create input parameter
dbutils.widgets.dropdown("PID Key Level 3",Level1_Allocation.columns[0], Level1_Allocation.columns)

dbutils.widgets.get("PID Key Level 3")

Out[272]: 'Company_Segment'

In [0]:
# Create input parameter
dbutils.widgets.dropdown("TXT Key Level 2",TXT_Only_df.columns[0], TXT_Only_df.columns)

dbutils.widgets.get("TXT Key Level 2")

Out[269]: 'Customer_number'

In [0]:
# Create input parameter
dbutils.widgets.dropdown("TXT Key Level 3",TXT_Only_df.columns[0], TXT_Only_df.columns)

dbutils.widgets.get("TXT Key Level 3")

Out[108]: 'Company_Segment'

In [0]:
#### For level 2 and higher, only allocate outstanding amount that not matched at level1 by key1, i.e. only exist in txt/invoice data ####
## Get subset from TXT/Invoice only that is used for level2  customer allocation

# Get key column to join from user input 
PID_key2 = getArgument('PID Key Level 2')
TXT_key2 = getArgument('TXT Key Level 2')
PID_key3 = getArgument('PID Key Level 3')
TXT_key3 = getArgument('TXT Key Level 3')

# Check if TXTonly data is empty, if not need to proceed to level2 customer allocation
Allocation_level = getArgument('Allocation Level')

if (Allocation_level == Level2 or Allocation_level == Level3) and TXT_Only_df.rdd.isEmpty() == True:
  print(f'\nTXTOnly is empty: {TXT_Only_df.rdd.isEmpty()}, No need to proceed to L2 allocation')
  sys.exit('exit')
elif (Allocation_level == Level2 or Allocation_level == Level3) and TXT_Only_df.rdd.isEmpty() == False:
  print(f'\nTXTOnly is empty: {TXT_Only_df.rdd.isEmpty()}, proceed to L2 allocation')
  # Get data for level2 customer allocation, by matching unallocationed invoice from level1 and txt/invoice only invoice 
  L2_Data = spark.sql("SELECT i.*, t.OA_TXT \
                       FROM (\
                             SELECT  {} \
                                    ,{} \
                                    ,SUM(OA)                        AS OA\
                                    ,SUM(Inscope_Carmel_OA_L1)      AS Inscope_Carmel_OA_L1\
                                    ,SUM(Inscope_Intersight_OA_L1)  AS Inscope_Intersight_OA_L1\
                                    ,SUM(Inscope_Adopter_OA_L1)     AS Inscope_Adopter_OA_L1\
                                    ,SUM(Inscope_TS_OA_L1)          AS Inscope_TS_OA_L1\
                                    ,SUM(Inscope_AS_OA_L1)          AS Inscope_AS_OA_L1\
                                    ,SUM(Outofscope_OA_L1)          AS Outofscope_OA_L1\
                             FROM L1_Allocation \
                             GROUP BY {}, {}) AS i \
                       INNER JOIN \
                                (SELECT {}, {}, SUM(OA) AS OA_TXT \
                                 FROM TXTOnly \
                                 GROUP BY {}, {}) AS t \
                       ON  i.{} == t.{} \
                       AND i.{} == t.{}".format(PID_key2,PID_key3,PID_key2,PID_key3,TXT_key2,TXT_key3,TXT_key2,TXT_key3,PID_key2,TXT_key2,PID_key3,TXT_key3))
else: 
  # if user not selects level2 or level3, end the script
  print(f'\nUser select allocation level is: {Allocation_level}, AR model has ended')
  sys.exit('exit')
        
L2_Data.createOrReplaceTempView("L2_Data")
L2_Data.show()

TXTOnly is empty: False, proceed to L2 allocation
+---------------+---------------+--------------------+--------------------+------------------------+---------------------+------------------+----------------+------------------+-------------------+
Customer_number|Company_Segment| OA|Inscope_Carmel_OA_L1|Inscope_Intersight_OA_L1|Inscope_Adopter_OA_L1| Inscope_TS_OA_L1|Inscope_AS_OA_L1| Outofscope_OA_L1| OA_TXT|
+---------------+---------------+--------------------+--------------------+------------------------+---------------------+------------------+----------------+------------------+-------------------+
 1020499234| 624| -22.14| 0.0| 0.0| 0.0| 0.0| 0.0| -22.14| -2197.2|
 1001760501| 020| 73754.32| 0.0| 0.0| 0.0| 0.0| 26049.0| 47705.31999999999| -3279.33|
 2927| 020| 3054862.79| 0.0| 0.0| 0.0| 0.0| 0.0| 3054862.79| -2111761.64|
 1028397580| 624| 5810.49| 0.0| 0.0| 0.0| 0.0| 0.0| 5810.49| -12712.24|
 1000872095| 624| 259001.02000000002| 0.0| 0.0| 0.0| 0.0| 0.0|259001.02000000002| -164114.72|
 112189212| 624| 7426227.4799999995| 826181.8319658916| 0.0| 75504.72512508077| 37689.71255669215| 0.0| 6486851.210352336|-1345981.0799999998|
 112605296| 624| 533697.85| 30131.74| 0.0| 22642.55| 0.0| 0.0| 480923.56|-467116.60000000003|
 407121888| 020| 1141511.1900000002| 0.0| 0.0| 0.0|21689.787800815197| 0.0| 1119821.402199185|-4403.4800000000005|
 42924| 020| 2664943.2199999997| -11654.380000000001| -10114.710000000001| 25340.84| 0.0| 0.0|2661371.4699999997|-125973.67000000001|
 112154709| 624| 747729.25| 62695.832946250826| 0.0| 81381.39915105604| 9682.845500000001| 0.0| 593969.1724026931| -37721.14|
 111431741| 555| 326511.0000000001| 0.0| 0.0| 0.0| 0.0| 0.0| 326511.0000000001|-365173.72000000003|
 1016230909| 020| 7169.0| 0.0| 0.0| 0.0| 0.0| 0.0| 7169.0| -178.0|
 1040526284| 198| 5.10178396E7| 0.0| 0.0| 0.0| 0.0| 0.0| 5.10178396E7| -21531.24|
 1029077998| 020|1.0738964439999998E7| 453090.19399564294| 28539.000000000007| 11273.82|1009892.7141000001| 0.0| 9236168.711904358| -7558.4|
 2000334472| 020| 570097.96| 78409.37791230205| 0.0| 0.0| 8338.5024| 0.0|483350.07968769793| -109790.66|
 286705| 671| 2259225.7499999995| 380459.39| 0.0| 1449.7167868865117|129690.34049999999| 0.0|1747626.3027131134| -71598.18|
 1035281814| 198| 191518.26999999996| 0.0| 0.0| 0.0| 0.0| 0.0|191518.26999999996|-147999.74000000002|
 1000046301| 624| 433365.4600000001| 146601.47| 0.0| 0.0| 6293.16| 0.0| 280470.83| -144594.48|
 113542181| 624| 204688.19999999998| 0.0| 0.0| 7450.63| 0.0| 0.0|197237.56999999998| -29795.58|
 1032054875| 624| 9058.92| 0.0| 0.0| 0.0| 0.0| 0.0| 9058.92| -762768.18|
+---------------+---------------+--------------------+--------------------+------------------------+---------------------+------------------+----------------+------------------+-------------------+
only showing top 20 rows

In [0]:
## Calculate customer percentage from matched invoices 
L2_Percent = spark.sql("SELECT   {}, {}\
                                ,SUM(OA_TXT)                                     AS OA_TXT \
                                ,SUM(Inscope_Carmel_OA_L1)/NULLIF(SUM(OA),0)     AS Inscope_Carmel_Percent_L2 \
                                ,SUM(Inscope_Intersight_OA_L1)/NULLIF(SUM(OA),0) AS Inscope_Intersight_Percent_L2 \
                                ,SUM(Inscope_Adopter_OA_L1)/NULLIF(SUM(OA),0)    AS Inscope_Adopter_Percent_L2 \
                                ,SUM(Inscope_TS_OA_L1)/NULLIF(SUM(OA),0)         AS Inscope_TS_Percent_L2 \
                                ,SUM(Inscope_AS_OA_L1)/NULLIF(SUM(OA),0)         AS Inscope_AS_Percent_L2 \
                                ,SUM(Outofscope_OA_L1)/NULLIF(SUM(OA),0)         AS Outofscope_Percent_L2 \
                          FROM L2_Data \
                          GROUP BY {}, {}".format(PID_key2,PID_key3,PID_key2,PID_key3))

L2_Percent.createOrReplaceTempView("L2_Percent")
L2_Percent.show()

+---------------+---------------+-------------------+-------------------------+-----------------------------+--------------------------+---------------------+---------------------+---------------------+
Customer_number|Company_Segment| OA_TXT|Inscope_Carmel_Percent_L2|Inscope_Intersight_Percent_L2|Inscope_Adopter_Percent_L2|Inscope_TS_Percent_L2|Inscope_AS_Percent_L2|Outofscope_Percent_L2|
+---------------+---------------+-------------------+-------------------------+-----------------------------+--------------------------+---------------------+---------------------+---------------------+
 2927| 020| -2111761.64| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
 1019446132| 624| -2607.74| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
 1071030| 020| -15220.5| 0.0| 0.0| 0.0| 0.004977554325054363| 0.0| 0.9950224456749458|
 1028397580| 624| -12712.24| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
 1001760501| 020|-3279.3300000000004| 0.0| 0.0| 0.0| 0.0| 0.35318609133675166| 0.6468139086632487|
 1008731933| 020| -1263.18| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
 1000872095| 624| -164114.72| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
 42924| 020|-125973.67000000001| -0.00437321887856...| -0.00379546923329...| 0.009508960569899123| 0.0| 0.0| 0.9986597275419624|
 112189212| 624| -1345981.08| 0.11125188855188313| 0.0| 0.010167305718607047| 0.005075216542746165| 0.0| 0.8735055891867639|
 407121888| 020|-4403.4800000000005| 0.0| 0.0| 0.0| 0.01900094190124864| 0.0| 0.9809990580987512|
 112154709| 624| -37721.14| 0.0838483086575132| 0.0| 0.1088380575603483| 0.01294966794464708| 0.0| 0.7943639658374916|
 112605296| 624|-467116.60000000003| 0.056458424930885515| 0.0| 0.04242578455206442| 0.0| 0.0| 0.90111579051705|
 15031| 020| -12000.0| 0.0| 0.0| 0.1493010007628935| 4.54151902104227E-7| 0.0| 0.8506985450852044|
 1040526284| 198| -21531.24| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
 1029077998| 020| -7558.4| 0.04219123701611241| 0.002657518810072...| 0.001049805133724...| 0.09404004638830893| 0.0| 0.8600613926517809|
 113542181| 624| -29795.58| 0.0| 0.0| 0.036399899945380336| 0.0| 0.0| 0.9636001000546196|
 286705| 671| -71598.18| 0.16840255561003586| 0.0| 6.41687439551586E-4| 0.05740477263062356| 0.0| 0.773550984319789|
 1013716514| 020| -86875.77000000002| 0.2844230596618313| 0.0| 0.0| 0.008974758304374716| 0.008375782747559108| 0.698226399286235|
 1018710641| 020| -195609.93| -0.0| -0.0| -0.0| -0.0| -0.0| 1.0|
 1000261412| 624| -68270.08| 0.0| 0.0| 0.0| 0.0| 0.0| 1.0|
+---------------+---------------+-------------------+-------------------------+-----------------------------+--------------------------+---------------------+---------------------+---------------------+
only showing top 20 rows

In [0]:
## Apply customer percentage to text only invoices outstanding amount
L2_Allocation = spark.sql("SELECT {} \
                                 ,{} \
                                 ,OA_TXT*IF(Inscope_Carmel_Percent_L2 IS NULL,0,Inscope_Carmel_Percent_L2)          AS Inscope_Carmel_OA_L2 \
                                 ,OA_TXT*IF(Inscope_Intersight_Percent_L2 IS NULL,0,Inscope_Intersight_Percent_L2)  AS Inscope_Intersight_OA_L2 \
                                 ,OA_TXT*IF(Inscope_Adopter_Percent_L2 IS NULL,0,Inscope_Adopter_Percent_L2)        AS Inscope_Adopter_OA_L2 \
                                 ,OA_TXT*IF(Inscope_TS_Percent_L2 IS NULL,0,Inscope_TS_Percent_L2)                  AS Inscope_TS_OA_L2 \
                                 ,OA_TXT*IF(Inscope_AS_Percent_L2 IS NULL,0,Inscope_AS_Percent_L2)                  AS Inscope_AS_OA_L2 \
                                 ,OA_TXT*IF(Outofscope_Percent_L2 IS NULL,0,Outofscope_Percent_L2)                  AS Outofscope_OA_L2 \
                                 ,OA_TXT                                                                            AS OA_TXT \
                          FROM L2_Percent".format(PID_key2,PID_key3))

L2_Allocation.createOrReplaceTempView("L2_Allocation")
L2_Allocation.show()

+---------------+---------------+--------------------+------------------------+---------------------+--------------------+------------------+-------------------+-------------------+
Customer_number|Company_Segment|Inscope_Carmel_OA_L2|Inscope_Intersight_OA_L2|Inscope_Adopter_OA_L2| Inscope_TS_OA_L2| Inscope_AS_OA_L2| Outofscope_OA_L2| OA_TXT|
+---------------+---------------+--------------------+------------------------+---------------------+--------------------+------------------+-------------------+-------------------+
 2927| 020| -0.0| -0.0| -0.0| -0.0| -0.0|-2111761.6399999997|-2111761.6399999997|
 1019446132| 624| -0.0| -0.0| -0.0| -0.0| -0.0|-2607.7400000000002|-2607.7400000000002|
 1071030| 020| -0.0| -0.0| -0.0| -75.76086560448994| -0.0|-15144.739134395513| -15220.5|
 1028397580| 624| -0.0| -0.0| -0.0| -0.0| -0.0| -12712.24| -12712.24|
 1001760501| 020| -0.0| -0.0| -0.0| -0.0| -1158.21374490335| -2121.116255096652|-3279.3300000000004|
 1008731933| 020| -0.0| -0.0| -0.0| -0.0| -0.0| -1263.18| -1263.18|
 1000872095| 624| -0.0| -0.0| -0.0| -0.0| -0.0| -164114.72| -164114.72|
 42924| 020| 550.9104318457487| 478.12918869081955| -1197.878660875484| -0.0| -0.0|-125804.83095966108| -125973.67|
 112189212| 624| -149742.9371051033| -0.0| -13685.001131820889| -6831.14544343935| -0.0| -1175721.996319637| -1345981.08|
 407121888| 020| -0.0| -0.0| -0.0| -83.67026764331037| -0.0| -4319.809732356689|-4403.4800000000005|
 112154709| 624| -3162.8537896332673| -0.0| -4105.495606561956| -488.4762374935448| -0.0|-29964.314366311235| -37721.14|
 112605296| 624| -26372.66749507048| -0.0| -19817.788232292856| -0.0| -0.0| -420926.1442726367|-467116.60000000003|
 15031| 020| -0.0| -0.0| -1791.6120091547218|-0.00544982282525...| -0.0|-10208.382541022453| -12000.0|
 1040526284| 198| -0.0| -0.0| -0.0| -0.0| -0.0| -21531.24| -21531.24|
 1029077998| 020| -318.898245862584| -20.086590174052205| -7.934847122745476| -710.7922866213942| -0.0| -6500.688030219221| -7558.4|
 113542181| 624| -0.0| -0.0| -1084.5561308145755| -0.0| -0.0|-28711.023869185425| -29795.58|
 286705| 671| -12057.316489027357| -0.0| -45.943652800753576| -4110.077243666458| -0.0|-55384.842614505425| -71598.18|
 1013716514| 020| -24709.472313877533| -0.0| -0.0| -779.6890382564478|-727.6525755469132| -60658.95607231912| -86875.77|
 1018710641| 020| 0.0| 0.0| 0.0| 0.0| 0.0| -195609.93| -195609.93|
 1000261412| 624| -0.0| -0.0| -0.0| -0.0| -0.0| -68270.08| -68270.08|
+---------------+---------------+--------------------+------------------------+---------------------+--------------------+------------------+-------------------+-------------------+
only showing top 20 rows

In [0]:
# Check allocation result
spark.sql("SELECT ROUND(SUM(Inscope_Carmel_OA_L2),2), ROUND(SUM(Inscope_Intersight_OA_L2),2), ROUND(SUM(Inscope_Adopter_OA_L2),2), ROUND(SUM(Inscope_TS_OA_L2),2), ROUND(SUM(Inscope_AS_OA_L2),2), ROUND(SUM(Outofscope_OA_L2),2) FROM L2_Allocation")

Out[58]:

"round(sum(Inscope_Carmel_OA_L2), 2)","round(sum(Inscope_Intersight_OA_L2), 2)","round(sum(Inscope_Adopter_OA_L2), 2)","round(sum(Inscope_TS_OA_L2), 2)","round(sum(Inscope_AS_OA_L2), 2)","round(sum(Outofscope_OA_L2), 2)"
-3.020286309E7,-1935259.66,-1.884350334E7,-5161001.22,-3356533.39,-1.07318410657E9


In [0]:
# Check allocation result at entity level
spark.sql("SELECT Company_Segment, ROUND(SUM(Inscope_Carmel_OA_L2),2), ROUND(SUM(Inscope_Intersight_OA_L2),2), ROUND(SUM(Inscope_Adopter_OA_L2),2), ROUND(SUM(Inscope_TS_OA_L2),2), ROUND(SUM(Inscope_AS_OA_L2),2), ROUND(SUM(Outofscope_OA_L2),2) FROM L2_Allocation GROUP BY Company_Segment ORDER BY Company_Segment ASC")

Out[59]:

Company_Segment,"round(sum(Inscope_Carmel_OA_L2), 2)","round(sum(Inscope_Intersight_OA_L2), 2)","round(sum(Inscope_Adopter_OA_L2), 2)","round(sum(Inscope_TS_OA_L2), 2)","round(sum(Inscope_AS_OA_L2), 2)","round(sum(Outofscope_OA_L2), 2)"
020,-4119767.85,-314922.59,-548107.65,-828378.68,-22410.72,-1.5735159843E8
105,6711547.78,165.04,-2649547.95,-112321.61,-758941.67,-2046702.34
115,-327914.03,-11572.43,-74524.66,-80896.47,-2151.75,-9002597.94
198,-1.091947217E7,-1296398.2,-6239691.5,-3467683.01,-2503633.06,-6.92922582E8
517,0.0,0.0,0.0,0.0,0.0,-589677.26
555,-637698.57,-897.69,-39980.42,-70569.08,0.11,-6208656.91
613,0.0,0.0,0.0,-54653.43,0.0,-1.716912993E7
624,-9982570.69,-319265.93,-903805.2,-522180.72,-67183.51,-5.153366323E7
671,-1105916.47,-19.38,-112708.87,26729.02,-1164.38,-1.251666579E7
696,-9816004.48,7651.52,-8274882.96,-51564.79,288.82,-1.2405668014E8


In [0]:
#### For level 2 and higher, only allocate outstanding amount that not matched at level1 by key1, i.e. only exist in txt/invoice data ####
## Get subset from TXT/Invoice only that is not matched at level2, then used for level3  entity allocation

# Check if all customer match between L2 allocation and TXTOnly, if not need to proceed to L3 entity allocation
Allocation_level = getArgument('Allocation Level')

L3_Check = spark.sql("SELECT * \
                      FROM TXTOnly AS t\
                      LEFT ANTI JOIN L2_Allocation AS l\
                      ON l.{} == t.{} AND l.{} == t.{}".format(PID_key2,TXT_key2,PID_key3,TXT_key3))

# Create L3 data for L3 entity allocation
if Allocation_level == Level3 and L3_Check.rdd.isEmpty() == True:
  print(f'\nTXTOnly after L2 allocation is empty: {L3_Check.isEmpty()}, No need to proceed to L3 allocation')
  sys.exit('exit')
elif Allocation_level == Level3 and L3_Check.rdd.isEmpty() == False:
  print(f'\nTXTOnly after L2 allocation is empty: {TXT_Only_df.rdd.isEmpty()}, proceed to L3 allocation')
  # Get data for level3 entity allocation, by matching unallocationed invoice from level2 and level1 and txt/invoice only invoice 
  L3_Data = spark.sql("SELECT      c.* \
                                  ,i.* \
                                  ,t.* \
                          FROM \
                          (SELECT {} \
                                  ,SUM(Inscope_Carmel_OA_L2) AS Inscope_Carmel_OA_L2 \
                                  ,SUM(Inscope_Intersight_OA_L2) AS Inscope_Intersight_OA_L2 \
                                  ,SUM(Inscope_Adopter_OA_L2) AS Inscope_Adopter_OA_L2 \
                                  ,SUM(Inscope_TS_OA_L2) AS Inscope_TS_OA_L2 \
                                  ,SUM(Inscope_AS_OA_L2) AS Inscope_AS_OA_L2 \
                                  ,SUM(Outofscope_OA_L2) AS Outofscope_OA_L2 \
                                  ,SUM(Inscope_Carmel_OA_L2)+SUM(Inscope_Intersight_OA_L2)+SUM(Inscope_Adopter_OA_L2)+SUM(Inscope_TS_OA_L2)+SUM(Inscope_AS_OA_L2)+SUM(Outofscope_OA_L2) AS OA_L2 \
                          FROM L2_Allocation AS c \
                          GROUP BY {}) AS c \
                          INNER JOIN \
                          (SELECT {} AS Key3_2 \
                                  ,SUM(Inscope_Carmel_OA_L1) AS Inscope_Carmel_OA_L1 \
                                  ,SUM(Inscope_Intersight_OA_L1) AS Inscope_Intersight_OA_L1 \
                                  ,SUM(Inscope_Adopter_OA_L1) AS Inscope_Adopter_OA_L1 \
                                  ,SUM(Inscope_TS_OA_L1) AS Inscope_TS_OA_L1 \
                                  ,SUM(Inscope_AS_OA_L1) AS Inscope_AS_OA_L1 \
                                  ,SUM(Outofscope_OA_L1) AS Outofscope_OA_L1 \
                                  ,SUM(Inscope_Carmel_OA_L1)+SUM(Inscope_Intersight_OA_L1)+SUM(Inscope_Adopter_OA_L1)+SUM(Inscope_TS_OA_L1)+SUM(Inscope_AS_OA_L1)+SUM(Outofscope_OA_L1) AS OA_L1 \
                           FROM L1_Allocation \
                           GROUP BY {}) AS i \
                          ON c.{} == i.Key3_2 \
                          INNER JOIN \
                          (SELECT {} AS Key3_3 \
                                  ,SUM(OA) AS OA_TXT \
                           FROM TXTOnly \
                           GROUP BY {}) AS t \
                          ON c.{} == t.Key3_3 \
                          ".format(PID_key3,PID_key3,PID_key3,PID_key3,PID_key3,PID_key3,TXT_key3,TXT_key3,PID_key3,TXT_key3))\
                .drop('Key3_2').drop('Key3_3')
else:
  # if user not selects level3, end the script
  print(f'\nUser select allocation level is: {Allocation_level}, AR model has ended')
  sys.exit('exit')
        
L3_Data.createOrReplaceTempView("L3_Data")
L3_Data.show()

TXTOnly after L2 allocation is empty: False, proceed to L3 allocation
+---------------+--------------------+------------------------+---------------------+------------------+-------------------+--------------------+--------------------+--------------------+------------------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
Company_Segment|Inscope_Carmel_OA_L2|Inscope_Intersight_OA_L2|Inscope_Adopter_OA_L2| Inscope_TS_OA_L2| Inscope_AS_OA_L2| Outofscope_OA_L2| OA_L2|Inscope_Carmel_OA_L1|Inscope_Intersight_OA_L1|Inscope_Adopter_OA_L1| Inscope_TS_OA_L1| Inscope_AS_OA_L1| Outofscope_OA_L1| OA_L1| OA_TXT|
+---------------+--------------------+------------------------+---------------------+------------------+-------------------+--------------------+--------------------+--------------------+------------------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 613| 0.0| 0.0| 0.0|-54653.43335192918| 0.0|-1.71691299266480...|-1.72237833600000...| 0.0| 0.0| 316635.8347541281| 4584288.7039| 1127419.3976000003| 3.988595017937459E8| 4.0488784573E8| -1.722378336E7|
 905| -5066.609015879238| 0.0| -254.11717155208981| 517.5625260533848|-1337.2390653203875| 213847.40272669835| 207707.00000000003| 7.837533918768022E8| 4381135.0| 3.2167897198446596E8| 7.825369247040713E7| 4.6823563E7|2.812104064566832...|2.935593140000000...| 73873.0|
 020| -4119767.8517845664| -314922.58685147075| -548107.6531237072|-828378.6795338311| -22410.71513982422|-1.57351598433566...|-1.63185185919999...|2.0745885449788192E8| 3.9008072971157E7| 3.0124160163814433E7| 3.354222098775688E7| 2722615.9878|3.3433599829315963E9|3.6562159075400066E9|-1.84133058529999...|
 198|-1.09194721695787...| -1296398.2023791177| -6239691.495709059|-3467683.013815336| -2503633.056394825| -6.92922582002123E8| -7.1734945994E8|1.2009048304125285E9| 6.75363780578031E7| 3.3927684384321725E8|1.9277421002486342E8| 7.03429571075E7|2.050626523466409...|2.237710045411000...|-8.467274677199997E8|
 517| 0.0| 0.0| 0.0| 0.0| 0.0| -589677.26| -589677.26| 0.0| 0.0| 0.0| 0.0| 0.0|1.8663345220000003E7|1.8663345220000003E7| -440852.76|
 115| -327914.02567574347| -11572.430151146178| -74524.66367585337|-80896.47468448669|-2151.7546696918075| -9002597.941143079| -9499657.290000001|1.6917985976863332E7| 583804.5361760929| 5198641.892673327| 2351410.155782155| 85042.61|2.7891850219850504E8|3.0405538736999995E8| -9578077.11|
 671| -1105916.4729518825| -19.379599830259995| -112708.8712282366|26729.021694472387|-1164.3820624797352|-1.25166657858520...|-1.37097458700000...| 4617633.98892361| 18461.64| 2376071.155308839| 3036279.9895951035| 428049.2|1.5714386753617245E8| 1.6762036351E8|-1.43261605700000...|
 555| -637698.567279775| -897.6870795661325| -39980.42220851959|-70569.07767111763|0.11417504056177959| -6208656.909936063| -6957802.550000001| 4014193.0076303496| 79484.84999999999| 1197805.288517069| 519857.7015586707| 778119.1916999999| 7.756927840059392E7| 8.415873844000001E7| -8675333.009999998|
 624| -9982570.691909827| -319265.933618488| -903805.204481988|-522180.7241252142| -67183.51023412535|-5.153366322563033E7|-6.332866928999997E7|1.0142185551459737E8| 5047449.044303749| 2.677132148730386E7|1.3980535248620473E7| 1271214.7118299138|1.9208479344033432E9|2.0693403104099987E9|-6.778347214999998E7|
 105| 6711547.782846194| 165.0386203169778| -2649547.953068428|-112321.6148362081| -758941.6704066847| -2046702.3431551931| 1144199.239999997|1.4685555321411453E7| 242825.14000000004| 4192252.8864645776| 3028184.320402234| 61810.042400000006|2.6071215601932174E8| 2.8292278373E8| 912834.7600000006|
 696| -9816004.482793601| 7651.520212553601| -8274882.957517535|-51564.78562706576| 288.82035049074335|-1.24056680144624...|-1.42191192030000...| 2.892804266649629E8| 3820486.44| 8.317275551638275E7| 3.526905356763711E7|1.1672361143000001E7| 5.7054034133180

In [0]:
# Replace null with 0 for all numeric columns
L3_Data.fillna(value=0)

L3_Data.createOrReplaceTempView("L3_Data")

In [0]:
## Calculate entity percentage from matched invoices and customers ##

L3_Percent = spark.sql("SELECT e.* \
                                  ,(Inscope_Carmel_OA_L1 + Inscope_Carmel_OA_L2)/NULLIF(OA_L1 + OA_L2,0)          AS Inscope_Carmel_Percent_L3 \
                                  ,(Inscope_Intersight_OA_L1 + Inscope_Intersight_OA_L2)/NULLIF(OA_L1 + OA_L2,0)  AS Inscope_Intersight_Percent_L3 \
                                  ,(Inscope_Adopter_OA_L1 + Inscope_Adopter_OA_L2)/NULLIF(OA_L1 + OA_L2,0)        AS Inscope_Adopter_Percent_L3 \
                                  ,(Inscope_TS_OA_L1 + Inscope_TS_OA_L2)/NULLIF(OA_L1 + OA_L2,0)                  AS Inscope_TS_Percent_L3 \
                                  ,(Inscope_AS_OA_L1 + Inscope_AS_OA_L2)/NULLIF(OA_L1 + OA_L2,0)                  AS Inscope_AS_Percent_L3 \
                                  ,(Outofscope_OA_L1 + Outofscope_OA_L2)/NULLIF(OA_L1 + OA_L2,0)                  AS Outofscope_Percent_L3 \
                          FROM L3_Data AS e ")

L3_Percent.createOrReplaceTempView("L3_Percent")
L3_Percent.show()

+---------------+--------------------+------------------------+---------------------+-------------------+-------------------+--------------------+--------------------+--------------------+------------------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------------+-----------------------------+--------------------------+---------------------+---------------------+---------------------+
Company_Segment|Inscope_Carmel_OA_L2|Inscope_Intersight_OA_L2|Inscope_Adopter_OA_L2| Inscope_TS_OA_L2| Inscope_AS_OA_L2| Outofscope_OA_L2| OA_L2|Inscope_Carmel_OA_L1|Inscope_Intersight_OA_L1|Inscope_Adopter_OA_L1| Inscope_TS_OA_L1| Inscope_AS_OA_L1| Outofscope_OA_L1| OA_L1| OA_TXT|Inscope_Carmel_Percent_L3|Inscope_Intersight_Percent_L3|Inscope_Adopter_Percent_L3|Inscope_TS_Percent_L3|Inscope_AS_Percent_L3|Outofscope_Percent_L3|
+---------------+--------------------+------------------------+---------------------+-------------------+-------------------+--------------------+--------------------+--------------------+------------------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------------+-----------------------------+--------------------------+---------------------+---------------------+---------------------+
 613| 0.0| 0.0| 0.0| -54653.43335192919| 0.0|-1.71691299266480...|-1.72237833600000...| 0.0| 0.0| 316635.8347541281| 4584288.7039| 1127419.3976000003|3.9885950179374593E8| 4.048878457300001E8| -1.722378336E7| 0.0| 0.0| 8.167789214671126E-4| 0.011684434308550451| 0.002908238103649525| 0.9845905486663329|
 905| -5066.609015879238| 0.0| -254.11717155208981| 517.5625260533848|-1337.2390653203875| 213847.40272669835| 207707.00000000003| 7.837533918768021E8| 4381135.0| 3.2167897198446596E8| 7.825369247040711E7| 4.6823563E7|2.812104064566832...|2.935593140000000...| 73873.0| 0.026697936074328677| 1.492408447865446...| 0.010957800570940536| 0.002665684671533436| 0.001594972199520947| 0.9579343656388898|
 020| -4119767.851784567| -314922.58685147075| -548107.6531237072| -828378.6795338311|-22410.715139824224|-1.57351598433566...|-1.63185185919999...|2.0745885449788186E8| 3.900807297115701E7| 3.012416016381444E7|3.3542220987756882E7| 2722615.9878000007| 3.343359982931594E9|3.6562159075400043E9|-1.84133058529999...| 0.05821279652295538| 0.011077243078572283| 0.008467160717376656| 0.00936546080334815| 7.730264884151577E-4| 0.9121043123893323|
 198|-1.091947216957876E7| -1296398.2023791177| -6239691.495709059| -3467683.013815336| -2503633.056394825| -6.92922582002123E8| -7.1734945994E8|1.2009048304125285E9| 6.753637805780312E7| 3.392768438432172E8|1.9277421002486342E8| 7.03429571075E7|2.050626523466409E10| 2.237710045411E10|-8.467274677199998E8| 0.05493993714716524| 0.003058205972601132| 0.015375853232899552| 0.008740014003946875| 0.003132045426993366| 0.9147539442163937|
 115| -327914.02567574347| -11572.430151146176| -74524.66367585337| -80896.47468448669|-2151.7546696918075| -9002597.941143077| -9499657.29|1.6917985976863332E7| 583804.5361760929| 5198641.892673327| 2351410.155782155| 85042.61|2.7891850219850504E8|3.0405538736999995E8| -9578077.110000001| 0.05632235348700161| 0.001942695549903...| 0.01739608741478493| 0.007708265191381638| 2.814097532843629...| 0.9163491886036441|
 671| -1105916.4729518825| -19.379599830259995| -112708.8712282366| 26729.021694472387|-1164.3820624797352|-1.25166657858520...|-1.37097458700000...| 4617633.988923611| 18461.64| 2376071.1553088385| 3036279.9895951045| 428049.2|1.5714386753617245E8| 1.6762036351E8| -1.432616057E7| 0.02281660336251594| 1.198244843855188...| 0.01470569294559425| 0.019901219670588395| 0.002773589142082...| 0.9396830703948335|
 555| -637698.567279775| -897.6870795661325| -39980.422208519594| -70569.07767111763|0.11417504056177961| -6208656.909936063| -6957802.550000001| 4014193.0076303505| 79484.8

In [0]:
## Apply entity percentage to text only invoices that not matched at level2 customer level
L3_Allocation = spark.sql("SELECT {} \
                                 ,(OA_TXT - OA_L2)*Inscope_Carmel_Percent_L3                     AS Inscope_Carmel_OA_L3 \
                                 ,(OA_TXT - OA_L2)*Inscope_Intersight_Percent_L3                 AS Inscope_Intersight_OA_L3 \
                                 ,(OA_TXT - OA_L2)*Inscope_Adopter_Percent_L3                    AS Inscope_Adopter_OA_L3 \
                                 ,(OA_TXT - OA_L2)*Inscope_TS_Percent_L3                         AS Inscope_TS_OA_L3 \
                                 ,(OA_TXT - OA_L2)*Inscope_AS_Percent_L3                         AS Inscope_AS_OA_L3 \
                                 ,(OA_TXT - OA_L2)*Outofscope_Percent_L3                         AS Outofscope_OA_L3 \
                           FROM L3_Percent".format(PID_key3))

L3_Allocation.createOrReplaceTempView("L3_Allocation")
L3_Allocation.show()

+---------------+--------------------+------------------------+---------------------+--------------------+--------------------+--------------------+
Company_Segment|Inscope_Carmel_OA_L3|Inscope_Intersight_OA_L3|Inscope_Adopter_OA_L3| Inscope_TS_OA_L3| Inscope_AS_OA_L3| Outofscope_OA_L3|
+---------------+--------------------+------------------------+---------------------+--------------------+--------------------+--------------------+
 613| 0.0| 0.0| 6.08547718425924E-12|8.705581954531708...|2.16680623862857E-11|7.335771237807966E-9|
 905| -3573.091576571705| -19.973499221162417| -1466.526281611256| -356.75924233000603| -213.46150935068647| -128204.18789091517|
 020| -1219434.2458346973| -232044.6768799319| -177369.00407599902| -196186.47984248152| -16193.260403476048|-1.910664494296301E7|
 198| -7108019.615658632| -395664.5961160303| -1989297.2591902101| -1130765.599799944| -405217.7976208598|-1.18349042911613...|
 517| 0.0| 0.0| 0.0| 0.0| 0.0| 148824.5|
 115| -4416.788822426845| -152.3458353382147| -1364.19804377164| -604.4807688203873| -22.068102198803185| -71859.93842744068|
 671| -14064.48971672436| -73.86157359515487| -9064.80530535067| -12267.40435287994| -1709.6811189400753| -579234.4579325147|
 555| -75118.67547286992| -1748.3705932418752| -25758.74828854171| -9995.564016968208| -17311.23585416983| -1587597.8657742026|
 624| -203061.62695897036| -10500.000704045922| -57444.67437560481| -29887.322185798173| -2673.8237856902215| -4151235.411989867|
 105| -17427.332037900956| -197.9085909027798| -1256.4893004349192| -2374.887259234635| 567.7938876210534| -210675.65669914728|
 696| -1739.840157023468| -23.83254404191082| -466.2859241116452| -219.25081101208585| -72.66951902692482| -34747.36104446568|
+---------------+--------------------+------------------------+---------------------+--------------------+--------------------+--------------------+

In [0]:
# Check allocation result
spark.sql("SELECT ROUND(SUM(Inscope_Carmel_OA_L3),2), ROUND(SUM(Inscope_Intersight_OA_L3),2), ROUND(SUM(Inscope_Adopter_OA_L3),2), ROUND(SUM(Inscope_TS_OA_L3),2), ROUND(SUM(Inscope_AS_OA_L3),2), ROUND(SUM(Outofscope_OA_L3),2) FROM L3_Allocation")

Out[64]:

"round(sum(Inscope_Carmel_OA_L3), 2)","round(sum(Inscope_Intersight_OA_L3), 2)","round(sum(Inscope_Adopter_OA_L3), 2)","round(sum(Inscope_TS_OA_L3), 2)","round(sum(Inscope_AS_OA_L3), 2)","round(sum(Outofscope_OA_L3), 2)"
-8646855.71,-640425.57,-2263487.99,-1382657.75,-442846.2,-1.4407041823E8


In [0]:
# Check allocation result at entity level
spark.sql("SELECT Company_Segment, ROUND(SUM(Inscope_Carmel_OA_L3),2), ROUND(SUM(Inscope_Intersight_OA_L3),2), ROUND(SUM(Inscope_Adopter_OA_L3),2), ROUND(SUM(Inscope_TS_OA_L3),2), ROUND(SUM(Inscope_AS_OA_L3),2), ROUND(SUM(Outofscope_OA_L3),2) FROM L3_Allocation GROUP BY Company_Segment ORDER BY Company_Segment ASC")

Out[65]:

Company_Segment,"round(sum(Inscope_Carmel_OA_L3), 2)","round(sum(Inscope_Intersight_OA_L3), 2)","round(sum(Inscope_Adopter_OA_L3), 2)","round(sum(Inscope_TS_OA_L3), 2)","round(sum(Inscope_AS_OA_L3), 2)","round(sum(Outofscope_OA_L3), 2)"
020,-1219434.25,-232044.68,-177369.0,-196186.48,-16193.26,-1.910664494E7
105,-17427.33,-197.91,-1256.49,-2374.89,567.79,-210675.66
115,-4416.79,-152.35,-1364.2,-604.48,-22.07,-71859.94
198,-7108019.62,-395664.6,-1989297.26,-1130765.6,-405217.8,-1.1834904291E8
517,0.0,0.0,0.0,0.0,0.0,148824.5
555,-75118.68,-1748.37,-25758.75,-9995.56,-17311.24,-1587597.87
613,0.0,0.0,0.0,0.0,0.0,0.0
624,-203061.63,-10500.0,-57444.67,-29887.32,-2673.82,-4151235.41
671,-14064.49,-73.86,-9064.81,-12267.4,-1709.68,-579234.46
696,-1739.84,-23.83,-466.29,-219.25,-72.67,-34747.36


In [0]:
# get cookies from browser to call sharepoint REST API
# these will expire every week or so
FedAuth = '77u/PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0idXRmLTgiPz48U1A+VjExLDBoLmZ8bWVtYmVyc2hpcHwxMDAzMjAwMTBjMTM0ZGNhQGxpdmUuY29tLDAjLmZ8bWVtYmVyc2hpcHxmb3J0dW5hLnpoYW5nQGV5LmNvbSwxMzI4MTY4MTY0MjAwMDAwMDAsMTMyODA1MDk2ODUwMDAwMDAwLDEzMjgyNjg0ODg2ODE5Njg0NywxMDguMTg1LjIxOC4xMTQsMzUsNWI5NzNmOTktNzdkZi00YmViLWIyN2QtYWEwYzcwYjg0ODJjLCxmMzAwNmY4Mi1lYjk1LTQ5MWItYTllZi1iMDk3NDc0YWNkZjcsNWIzNjA0YTAtNjBjNS0xMDAwLTI5ODctZWMzODM1MzQzYzI4LGQ4NTYwNmEwLTAwMTItMTAwMC0zNWRjLWFkYWNmY2EzNTUxOSwsMCwxMzI4MjI1NjQ4NjgwNDA2MTksMTMyODI1MTIwODY4MDQwNjE5LCwsZXlKNGJYTmZZMk1pT2lKYlhDSkRVREZjSWwwaUxDSndjbVZtWlhKeVpXUmZkWE5sY201aGJXVWlPaUpHYjNKMGRXNWhMbHBvWVc1blFHVjVMbU52YlNKOSwyNjUwNDY3NzQzOTk5OTk5OTk5LDEzMjgxNjgxOTUxMDAwMDAwMCxjODM0NWViNy1lMTllLTRkMzItYWM5MS01MmY2MWZjZjFmOGQsLCwsLCwwLEF6Z2c3MDl5cDB2ekRGc3pmVjd6bWpUWVhpZFB6VmRhcWVwTUk1WWFlSlZRcEVCRHpSYWpHUndKVHpkdHR6YitXUFF0MHNCbkFWT1BzdUFHQkdhKzl5QlAzMUh3U1UveWJ2dUdKbEhUS0sxYkZvOG1oemtnSFNvQ000WjhXbzhHWFIxRnROT21BREJ6cHh2MW1oR0dtUHZyVGFHWktwLy9hNVBuSU16VGFnbllvOURkN0NuWDlrNW9qdVhCNTBFdkhNMUdpUUZESkx4b09YZnZleDErdjhjc01ucEpGdHJYWU1MbVhJblNFVTZYb3U0bDNNWHJVZHRDblA1T2tRS1lLT1JmVTlRWCtvcTc4ZzhVOUEwY0F0RHB6Vk1jTm5NWnloN3JJTlBySVV5czZuNmozYjVKNzFsc1VLUlA1elMrQWJ1Q1UzVGVHV3hraU5wcXZRQ1hCZz09PC9TUD4='

rtFA = 'yPOPU0EwzrVOSUlLAtfyKEa1mTMrE6eCg/ghSF7Qx74mNUI5NzNGOTktNzdERi00QkVCLUIyN0QtQUEwQzcwQjg0ODJDIzEzMjgxNjgxOTUxOTI0NTQwOCM1QjM2MDRBMC02MEM1LTEwMDAtMjk4Ny1FQzM4MzUzNDNDMjgjRk9SVFVOQS5aSEFORyU0MEVZLkNPTVZOjIv99EiZhB6Zu2hCemqdSZfh2GuHwvWGGtV+yG/5RYt6RKnKySUd8UskxqdPpvcGcYFBOlbVYBxPn60t1iQaXsPFqBId/8abPKfCdiwGcBtzzIFTEMDGWQ1M+qiaQ4xoQhiAr+WR6vvJKQGEJMq/2NpLD932Rg4DNqjWrPeIheo+F8Dtz+f/lwQe2RODFgLfuK2gHC/T3JyJIScgHZck43OV3rp5cdnbqTyp0LjQ9uVWAw8dn9U96T13ID6SwcsxBZgssI717FmyJS+F8Uul+PetLkCbsyNOEWYp3C0UqahviKZAStaJ3VGLpyG+itQ1TOPDFE18t2IYzK5+mDCUAAAA'

In [0]:
import requests
import json
import time
import numpy as np
import re
import pandas as pd

cookies = {
  'FedAuth': FedAuth,
  'rtFA': rtFA
}

headers = {
  'Accept': 'application/json; odata=verbose'
}

# get metadata for all the documents in the document library
uri = "https://eyus.sharepoint.com/sites/ProjectG360Confidential/_api/web/lists/getbytitle('Documents')/items?" + \
  "$select=File/Name,File/ServerRelativeUrl,File/Length,Modified" + \
  "&$expand=File" + \
  "&$top=5000" + \
  "&$skiptoken=Page=TRUE"

  
full_results = []
has_next_results = True

while has_next_results:
  r = requests.get(uri, headers=headers, cookies=cookies)
  if r.status_code == 200:   
    response = r.json()
    items = response['d']['results']
    full_results.extend(items) # add result set to array
    if '__next' in response['d']:
      uri = response['d']['__next']
      time.sleep(1) # small delay to avoid throttling on next page
    else:
      has_next_results = False
  else:
    # error getting response
    print(r.status_code)
    print(r.text)
    has_next_results = False

# filter array for only file type results (exclude folders or other list items)
file_results = [{'ItemUri': item['__metadata']['uri'], 'ServerRelativeUrl': item['File']['ServerRelativeUrl'], 'Name': item['File']['Name'], 'Length': item['File']['Length'], 'Modified': item['Modified']} for item in full_results if 'ServerRelativeUrl' in item['File']]


In [0]:
# filter only for AR files
AR_file_prefix = '/sites/ProjectG360Confidential/Shared Documents/General/Data Analysis/AR/PID Analysis/FY 21'

for item in file_results:
  if item['ServerRelativeUrl'].startswith(AR_file_prefix):
    print(item['Name'])
#   else:
#     print(item['ServerRelativeUrl'])

AR Workflow - 1.1 Combined Part 1 FY21.yxmd
AR Workflow - 3 Output FY21.yxmd
AR Workflow - 2 TXT FY21.yxmd
AR Workflow - 4 Compare FY21.yxmd
AR Workflow - TXT v5 - FY21.bak
AR Workflow - Output v5 - FY21.bak
AR Workflow - Compare v5 - FY21.bak
AR Workflow - Output v5.bak
AR Workflow - TXT FY21.bak
AR Workflow - Compare FY21.bak
AR Workflow -PID Allocation FY21.bak
AR Workflow - 1.2 Combined Part 2 FY21.yxmd
AR PID Data - Combined Part 1 FY21.csv
AR Workflow - 1.1 Combined Part 1 FY21.bak
AR PID Data - Combined Part 2 FY21.csv
AR Workflow - 1.2 Combined Part 2 FY21.bak
AR Workflow - 3.2 Output Part 2 FY21.yxmd
AR Workflow - 3.1 Output Part 1 FY21.yxmd
AR Workflow - 4 Compare FY20.bak
AR Workflow - 3.2 Output Part 2 FY21.bak
AR Workflow - 3.1 Output Part 1 FY21.bak
AR Workflow - 5.1 PID Allocation Prep FY21.yxmd
AR Workflow - 5.2 PID Allocation FY21 v1.yxmd
AR Workflow - 4 Compare FY21.bak
PID Allocation Prep FY21.csv
AR Workflow - 5.2 PID Allocation v2 FY21.bak
AR Workflow - 5.1 PID Allocation Prep FY21.bak
AR Workflow - 2 TXT FY21.bak
AR PID Data - PID Allocation FY21.xlsx
AR PID Data - PID Invoice Review FY21.xlsx
AR PID Data - Output Part 1 FY21.xlsx
AR PID Data - Output Part 2 FY21.xlsx
AR PID Data - Output Total FY21.xlsx
AR PID Data - Compare FY21.xlsx
AR Workflow - 2 TXT FY21 v2.yxmd
AR Workflow - 4 Compare FY21 v2.yxmd
AR Workflow - 2 TXT FY21 v2.bak
AR PID Data - Compare FY21 v2.xlsx
AR PID Data - TXT FY21.xlsx
AR Workflow - 4 Compare FY21 v2.bak
AR PID Data - TXT FY21 v2.xlsx.bak
AR PID Data - Compare FY21.xlsx
AR Workflow - 3.2 Output Part 2 FY21 v2.yxmd
AR PID Data - TXT FY21 v2.xlsx
AR Workflow - 3.1 Output Part 1 FY21 v2.yxmd
AR Workflow - 3.1 Output Part 1 FY21 v2.bak
AR Workflow - 3.2 Output Part 2 FY21 v2.bak
AR Workflow - 1.3 Combined Part 3 FY21.yxmd
AR PID Data - Combined Part 3 FY21.csv
AR Workflow - 3.3 Output Part 3 FY21 v2.yxmd
AR Workflow - 1.3 Combined Part 3 FY21.bak
AR Workflow - 3.3 Output Part 3 FY21 v2.bak
AR Workflow - 5.2 PID Allocation FY21 v2.yxmd
AR Workflow - 5.1 PID Allocation Prep FY21 v2.yxmd
PID Allocation Prep FY21 v2.csv
AR PID Data - Compare FY21 v2.xlsx-US3100890W1.bak
AR Workflow - 5.1 PID Allocation Prep FY21 v2.bak
AR Workflow - 5.2 PID Allocation FY21 v2.bak
AR Workflow - 4 Compare FY21Q1.yxmd
AR Workflow - 3.1 Output Part 1 FY21Q1.yxmd
AR Workflow - 2 TXT FY21Q1.yxmd
AR Workflow - 1.1 Combined Part 1 FY21Q1.yxmd
AR Workflow - 5.2 PID Allocation FY21Q1.yxmd
AR Workflow - 5.1 PID Allocation Prep FY21Q1.yxmd
AR PID Data - Combined Part 1 FY21Q1.csv
AR Workflow - 2 TXT FY21Q1.bak
AR Workflow - 1.1 Combined Part 1 FY21Q1.bak
AR Workflow - 3.1 Output Part 1 FY21Q1.bak
AR Workflow - 2 TXT FY21Q2.yxmd
AR Workflow - 1.1 Combined Part 1 FY21Q2.yxmd
AR Workflow - 4 Compare FY21Q1.bak
AR Workflow - 4 Compare FY21Q2.yxmd
AR Workflow - 3.1 Output Part 1 FY21Q2.yxmd
AR Workflow - 4 Compare FY21Q3.yxmd
AR Workflow - 3.1 Output Part 1 FY21Q3.yxmd
AR Workflow - 2 TXT FY21Q3.yxmd
AR Workflow - 1.1 Combined Part 1 FY21Q3.yxmd
AR PID Data - Combined Part 1 FY21Q2.csv
AR Workflow - 1.1 Combined Part 1 FY21Q2.bak
AR PID Data - Combined Part 1 FY21Q3.csv
AR Workflow - 1.1 Combined Part 1 FY21Q3.bak
AR Workflow - 2 TXT FY21Q2.bak
AR Workflow - 2 TXT FY21Q3.bak
AR Workflow - 3.1 Output Part 1 FY21Q2.bak
AR Workflow - 3.1 Output Part 1 FY21Q3.bak
AR Workflow - 4 Compare FY21Q2.bak
AR Workflow - 4 Compare FY21Q3.bak
AR Workflow - 5.1 PID Allocation Prep FY21Q1.bak
AR PID Data - TXT FY21Q2.xlsx.bak
AR PID Data - TXT FY21Q3.xlsx
PID Allocation Prep FY21Q1.csv
AR PID Data - Summary FY21Q1.xlsx-US3100890W1.bak
AR PID Data - Summary FY21Q1.xlsx-US3100890W1-2.bak
AR PID Data - TXT FY21Q1.xlsx-US3100890W1.bak
AR PID Data - TXT FY21Q1.xlsx.bak
AR PID Data - PID Allocation FY21Q1.xlsx.bak
AR PID Data - Summary FY21Q1.xlsx-US3100890W1-3.bak
AR Workflow - 5.2 PID Allocation FY21Q2.yxmd
AR Workflow - 5.1 PID Allocation Prep FY21Q2.bak
AR Workflow - 5.2 PID Allocation FY21Q1.bak
PID Allocation Prep FY21Q2.csv
AR PID Da

In [0]:
########WIP

# download the files and put them in folder structure at /CACL/<Category>/<Year, if applicable>/<Filename>
# note: this sync approach assumes that files may be replaced (as shown by file modified date), but need to take care not to duplicate data
# e.g. cannot have two files US80_V1.txt and US80_V2.txt in the raw data files or else this script will read both files and therefore have duplicate data

import os
from datetime import datetime, timezone
from dateutil import parser
import urllib


def DownloadFBL3N(file, i, download_df):
  
  save_file_dir = '/dbfs/mnt/files/ir/files/AR'

  file_uri_prefix = 'https://eyus.sharepoint.com/sites/PraojectG360Confidential/_layouts/download.aspx?SourceUrl=https://eyus.sharepoint.com'

  df_len = len(download_df)
 
  file_download_uri = f'{file_uri_prefix}{urllib.parse.quote(file.ServerRelativeUrl)}'
  
  # generate path to save at
#   if file.Category == 'P&L':
#     save_file_category_dir = f'{save_file_dir}/PL'
#   elif file.Category == 'Missing Docs':
#     save_file_category_dir = f'{save_file_dir}/Missing Docs'
#   elif file.Category == 'BS Posting Date Items':
#     save_file_category_dir = f'{save_file_dir}/BS Posting Date Items'
#   elif file.Category == 'BS Open Items':
#     save_file_category_dir = f'{save_file_dir}/BS Open Items/{file.Year}'
#   else:
#     save_file_category_dir = f'{save_file_dir}/Unknown'    
    
  # ensure the directory exists - need to remove /dbfs when creating directories
  dbutils.fs.mkdirs(save_file_category_dir.replace('/dbfs', ''))
  
  # check if the file exists already and whether it has been updated  
  save_file_path = f'{save_file_category_dir}/{file.Name}'
  file_already_saved = False
  try:
    file_already_saved = True
    file_stat = os.stat(save_file_path)
    sharepoint_modified = parser.isoparse(file.Modified)
    file_modified = datetime.fromtimestamp(file_stat.st_mtime, tz=timezone.utc)
  except FileNotFoundError:
    file_already_saved = False
  
  if file_already_saved and sharepoint_modified <= file_modified:
    # file is the same so skip this file
    print(f'{i+1} of {df_len} | Skipping unmodified {file.Name}, Last saved: {file_modified}, Last modified in SP: {sharepoint_modified}')
  else:
    # download the file
    file_request = requests.get(file_download_uri, cookies=cookies)
  
    if file_request.status_code == 200:
      with open(save_file_path, 'wb') as out_file:
        out_file.write(file_request.content)
      print(f'{i+1} of {df_len} | Successfully downloaded {file.Name}')
    else:
      print(f'{i+1} of {df_len} | Failed to download {file.Name}')

In [0]:
# filter only for AR files
file_results = [item for item in file_results if item['ServerRelativeUrl'].startswith(AR_file_prefix) and item['Name'].lower().endswith('.csv')]

# put results into a data frame
sp_files_df = pd.DataFrame(file_results)